In [67]:
#General
import numpy as np
import pandas as pd

#Data Retrieval and Visualization
import pandas_datareader.data as web
import datetime
import matplotlib.pyplot as plt
%matplotlib inline

#Statistical Package
from scipy import stats 
import statsmodels.api as sm
import math
from scipy.stats import linregress
from scipy.stats import f

In [68]:
from pandas_datareader.famafrench import get_available_datasets
#spy_etf=web.DataReader('SPY','yahoo','2010-01-01','2020-01-01' ) 
beg1='2015-01-01' #'1963-07-01'
end1='2020-09-01' #'2014-12-01'

In [69]:
def fm_summary(p): #Summary Statistics Function
    s = p.describe().T
    s['std_error'] = s['std']/np.sqrt(s['count'])
    s['tstat'] = s['mean']/s['std_error']
    return s[['mean','std','tstat']].T
def update_withrf(dataport):
    for x in range(len(dataport.columns)):
        dataport.iloc[:,x]=dataport.iloc[:,x]-dataport['RF']
    dataport.drop('RF',axis=1,inplace=True)
        

In [70]:
len(get_available_datasets())
#print(ds['DESCR'])
datasets = get_available_datasets()
datasets

['F-F_Research_Data_Factors',
 'F-F_Research_Data_Factors_weekly',
 'F-F_Research_Data_Factors_daily',
 'F-F_Research_Data_5_Factors_2x3',
 'F-F_Research_Data_5_Factors_2x3_daily',
 'Portfolios_Formed_on_ME',
 'Portfolios_Formed_on_ME_Wout_Div',
 'Portfolios_Formed_on_ME_Daily',
 'Portfolios_Formed_on_BE-ME',
 'Portfolios_Formed_on_BE-ME_Wout_Div',
 'Portfolios_Formed_on_BE-ME_Daily',
 'Portfolios_Formed_on_OP',
 'Portfolios_Formed_on_OP_Wout_Div',
 'Portfolios_Formed_on_OP_Daily',
 'Portfolios_Formed_on_INV',
 'Portfolios_Formed_on_INV_Wout_Div',
 'Portfolios_Formed_on_INV_Daily',
 '6_Portfolios_2x3',
 '6_Portfolios_2x3_Wout_Div',
 '6_Portfolios_2x3_weekly',
 '6_Portfolios_2x3_daily',
 '25_Portfolios_5x5',
 '25_Portfolios_5x5_Wout_Div',
 '25_Portfolios_5x5_Daily',
 '100_Portfolios_10x10',
 '100_Portfolios_10x10_Wout_Div',
 '100_Portfolios_10x10_Daily',
 '6_Portfolios_ME_OP_2x3',
 '6_Portfolios_ME_OP_2x3_Wout_Div',
 '6_Portfolios_ME_OP_2x3_daily',
 '25_Portfolios_ME_OP_5x5',
 '25_Portf

In [71]:
#Extracting data
betaPort=web.DataReader('25_Portfolios_ME_BETA_5x5','famafrench',start=beg1,end=end1)
niPort=web.DataReader('25_Portfolios_ME_NI_5x5','famafrench',start=beg1,end=end1)
varPort=web.DataReader('25_Portfolios_ME_VAR_5x5','famafrench',start=beg1,end=end1)
rVarPort=web.DataReader('25_Portfolios_ME_RESVAR_5x5','famafrench',start=beg1,end=end1)
acPort=web.DataReader('25_Portfolios_ME_AC_5x5','famafrench',start=beg1,end=end1)
momPort=web.DataReader('25_Portfolios_ME_Prior_12_2','famafrench',start=beg1,end=end1)
FF5=web.DataReader('F-F_Research_Data_5_Factors_2x3','famafrench',start=beg1,end=end1)
MOM=web.DataReader('F-F_Momentum_Factor','famafrench',start=beg1,end=end1)


In [72]:
betaPort

{0:          SMALL LoBETA  ME1 BETA2  ME1 BETA3  ME1 BETA4  SMALL HiBETA  \
 Date                                                                   
 2015-01       -5.3658    -4.7388    -4.3938    -4.8308       -5.0361   
 2015-02        4.0822     3.1678     6.6206     6.2888        7.3418   
 2015-03        3.0534     1.9472     0.7722     1.0416       -0.7106   
 2015-04       -1.7786    -2.3128    -2.8453    -1.8458       -1.6802   
 2015-05        1.4652     2.2770    -0.5251     0.4948        1.4235   
 ...               ...        ...        ...        ...           ...   
 2020-05        5.6400     1.5617     3.1312     5.5310       12.1763   
 2020-06        8.0388     2.8686     6.1532     9.8156       10.7254   
 2020-07        3.0499     0.1371    -1.1180     2.6166        6.2377   
 2020-08        1.9596     5.5232     7.8214    12.4467        3.4481   
 2020-09       -2.3503    -3.7621    -3.2894    -1.0476       -2.4456   
 
          ME2 BETA1  ME2 BETA2  ME2 BETA3  ME2

In [73]:
FF5[0]
FF6=pd.concat([FF5[0],MOM[0]],axis=1,sort=False)

In [74]:
FF6.columns

Index(['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'RF', 'Mom   '], dtype='object')

## Table 1

In [75]:
Table1=fm_summary(FF6).copy()
Table1.rename(str.upper,axis='columns',inplace=True)
Table1.drop('RF',axis=1)

,MKT-RF,SMB,HML,RMW,CMA,MOM
mean,0.915652,-0.303768,-0.883768,0.174783,-0.327246,0.472899
std,4.413564,2.635677,3.108918,1.464596,1.627340,3.913689
tstat,1.723319,-0.957359,-2.361313,0.991300,-1.670403,1.003705


## Table 2 

In [76]:
#Excess return of each anomaly portfolio
betaPortXs = betaPort[0].sub(FF5[0]['RF'],axis=0)
niPortXs = niPort[0].sub(FF5[0]['RF'],axis=0)
varPortXs = varPort[0].sub(FF5[0]['RF'],axis=0)
rVarPortXs = rVarPort[0].sub(FF5[0]['RF'],axis=0)
acPortXs = acPort[0].sub(FF5[0]['RF'],axis=0)
momPortXs = momPort[0].sub(FF5[0]['RF'],axis=0)


In [77]:
def GRS(R,F): #GRS Statistics
    R=R.astype('float64')
    F=F.astype('float64')
    alphas = []
    alpha2=[]
    alphasquared=[]
    errors = []
    betas=[]
    returns=[]
    risquared=[]
    rsquared=[]
    bsealphas=[]
    for p in R.columns:
        model = sm.OLS(R[:][p], sm.add_constant(F[:]))
        fit = model.fit()
        alphas.append(fit.params["const"])
        alpha2.append(abs(fit.params["const"]))
        alphasquared.append(abs(fit.params["const"])**2)
        risquared.append((R[:][p].mean()-F['Mkt-RF'].mean())**2)
        betas.append(fit.params[1:])
        errors.append(fit.resid)
        returns.append(abs(R[:][p].mean()-F['Mkt-RF'].mean()))
        
        rsquared.append(fit.rsquared_adj)
        bsealphas.append(fit.bse["const"]**2)
   #For remaining statistics to compute mean. Convert list to arrasy
    alpha2=pd.np.array(alpha2)
    alphasquared=pd.np.array(alphasquared)
    returns=pd.np.array(returns)
    rsquared=pd.np.array(rsquared)
    bsealphas=pd.np.array(bsealphas)
    risquared=pd.np.array(risquared)
    #For GRS
    alphas = pd.np.array(alphas)
    errors = pd.np.array(errors).transpose()
    Ri=returns.mean()
    K = F.shape[1]
    T, N = errors.shape

    Ve = errors.transpose().dot(errors) / (T - K - 1)#unbiased estimate of the covariance matrix of residuals

    r = F.mean().values #step 5Factor mean vector

    f = F.values

    Vf = (f-r).transpose().dot(f-r) / (T - 1) #Step 7 unbiased estimate of the covariance matrix of the factors

    Sh2_a = alphas.dot(pd.np.linalg.pinv(Ve)).dot(alphas)
    Sh2_f = r.dot(pd.np.linalg.pinv(Vf)).dot(r)
    Sh2_a, Sh2_f
    ans=(T/N)*((T-N-K)/(T-K-1))*(Sh2_a/(1+Sh2_f))
    ans=ans.astype(float)
    #print("P-value of the GRS statistics: ",(1-f.cdf(ans,N,T-N-K)))
    #print("Average Absolute value of intercept A|ai|: ",alpha2.mean())
    #print("Average Absolute value of intercept/return A|ai|/A|ri| : ",alpha2.mean()/Ri)
    #print("Average Absolute value of intercept/return squared: ",(alpha2.mean()**2/Ri**2))
    #print("Average Absolute value of the As^2(ai)/A(ai)^2: ",(bsealphas.mean()))
    #print("Average value of the regression R^2 corrected for degress of freedom: ",rsquared.mean())
    return (ans,N,T-N-K,alpha2.mean(),alpha2.mean()/Ri,(alphasquared.mean()/risquared.mean()),(bsealphas.mean()/alphasquared.mean()),rsquared.mean())

In [78]:
#Factors
F1=FF5[0][['Mkt-RF']] #For portfolio beta only
F2=FF5[0][['Mkt-RF','SMB','HML']]
F3=FF5[0][['Mkt-RF','SMB','HML','RMW']]
F4=FF5[0][['Mkt-RF','SMB','HML','CMA']]
F5=FF5[0][['Mkt-RF','SMB','RMW','CMA']]
F6=FF5[0][['Mkt-RF','SMB','HML','RMW','CMA']]
#For Momentum Anomaly only
F71=FF6[['Mkt-RF','SMB','HML','Mom   ']]
F72=FF6[['Mkt-RF','SMB','HML','RMW','Mom   ']]
F73=FF6[['Mkt-RF','SMB','HML','CMA','Mom   ']]
F74=FF6[['Mkt-RF','SMB','RMW','CMA','Mom   ']]
F75=FF6[['Mkt-RF','SMB','HML','RMW','CMA','Mom   ']]

In [79]:
result=GRS(betaPortXs,F1)
#1-f.cdf(GRS(betaPortXs,F1)[0],GRS(betaPortXs,F1)[1],GRS(betaPortXs,F1)[2])

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:27: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:30: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:31: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directl

In [80]:
#Index name for GRS-Statistics
def indexname(f):
    name=""
    for data in list(f.columns):
        name=name+str(data)+" "
    return name
#Initialize
factorList=[F1,F2,F3,F4,F5,F6]
momfactorlist=[F2,F3,F4,F5,F6,F71,F72,F73,F74,F75]
portfolioList=[betaPortXs,niPortXs,varPortXs,rVarPortXs,acPortXs]
portnameList=["35 Size-NI Portffolio","25 Size-Var Portfolio","25 Size-RVar portfolios","25 Size-AC portfolios"] 
column_names=["Model Factors" , "GRS","p(GRS)" ,"A|ai|" ,"A|ai|/A|ri|","Aai^2/Ari^2","As^2(ai)/Aasi^2","A(R^2)"]
df=pd.DataFrame(columns=column_names)
counter=0

In [81]:
df.loc[0]=['25 Size Beta Portfolio',"","","","","","",""]
for fa in factorList:
    ans=GRS(betaPortXs,fa)
    counter+=1
    df.loc[counter]=[indexname(fa),ans[0],1-f.cdf(ans[0],ans[1],ans[2]),ans[3],ans[4],ans[5],ans[6],ans[7]]  

for port,name in list(zip(portfolioList[1:],portnameList)):
    counter+=1
    df.loc[counter]=[name,"","","","","","",""]
    for fa in factorList[1:]:
            ans=GRS(port,fa)
            counter+=1
            df.loc[counter]=[indexname(fa),ans[0],1-f.cdf(ans[0],ans[1],ans[2]),ans[3],ans[4],ans[5],ans[6],ans[7]]  
counter+=1       
df.loc[counter]=['25 Size Prior2-12 portfolios',"","","","","","",""]
for fa in momfactorlist:
    ans=GRS(momPortXs,fa)
    counter+=1
    df.loc[counter]=[indexname(fa),ans[0],1-f.cdf(ans[0],ans[1],ans[2]),ans[3],ans[4],ans[5],ans[6],ans[7]]  

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:27: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:30: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:31: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directl

In [82]:
df.set_index('Model Factors',drop=True,inplace=True)
#print (df.to_string(index=False))

In [83]:
df

,GRS,p(GRS),A|ai|,A|ai|/A|ri|,Aai^2/Ari^2,As^2(ai)/Aasi^2,A(R^2)
Model Factors,,,,,,,
25 Size Beta Portfolio,,,,,,,
Mkt-RF,1.35713,0.185777,0.46016,1.46524,2.10761,0.329316,0.811002
Mkt-RF SMB HML,1.06218,0.422179,0.172798,0.550221,0.377561,0.775809,0.919055
Mkt-RF SMB HML RMW,1.02302,0.463927,0.171065,0.544702,0.35791,0.784375,0.921889
Mkt-RF SMB HML CMA,1.04318,0.442601,0.174118,0.554423,0.383836,0.754636,0.919826
Mkt-RF SMB RMW CMA,1.15875,0.331627,0.159966,0.509362,0.303252,0.956615,0.915829
Mkt-RF SMB HML RMW CMA,1.00443,0.484586,0.172117,0.548053,0.363182,0.762639,0.922715
35 Size-NI Portffolio,,,,,,,
Mkt-RF SMB HML,1.06441,0.432348,0.216841,0.590152,0.39352,1.09817,0.852404


### 2.3 The five factor model vs FF three factor model

In [84]:
RMWexposure=GRS(FF5[0][['RMW']],F2)
CMAexposure=GRS(FF5[0][['CMA']],F2)


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:27: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:30: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:31: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directl

In [85]:
RMWexposure #return (grs,N,T-N-K,alpha2.mean(),alpha2.mean()/Ri,(alphasquared.mean()/risquared.mean()),(bsealphas.mean()/alphasquared.mean()),rsquared.mean())
CMAexposure

(0.05836992557375684,
 1,
 65,
 0.04131567127106892,
 0.03324138663367252,
 0.0011049897853293023,
 17.167843788518326,
 0.3461089424444507)

In [86]:
modelCMA = sm.OLS(FF5[0][['CMA']], sm.add_constant(F2))
modelCMA.fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    CMA   R-squared:                       0.375
Model:                            OLS   Adj. R-squared:                  0.346
Method:                 Least Squares   F-statistic:                     13.00
Date:                Sat, 21 Nov 2020   Prob (F-statistic):           9.50e-07
Time:                        22:34:51   Log-Likelihood:                -114.79
No. Observations:                  69   AIC:                             237.6
Df Residuals:                      65   BIC:                             246.5
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0413      0.171      0.241      0.810      -0.301       0.383
Mkt-RF        -0.1155      0.039     -2.933      0.005      -0.194      -0.037
SMB           -0.0538      0.070     -0.769      0.445      -0.194       0.086
HML            0.3158      0.056      5.671      0.000       0.205       0.427
==============================================================================
Omnibus:                        0.459   Durbin-Watson:                   2.283
Prob(Omnibus):                  0.795   Jarque-Bera (JB):                0.148
Skew:                           0.100   Prob(JB):                        0.928
Kurtosis:                       3.109   Cond. No.                         5.10
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [87]:
modelRMW = sm.OLS(FF5[0][['RMW']], sm.add_constant(F2))
modelRMW.fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    RMW   R-squared:                       0.132
Model:                            OLS   Adj. R-squared:                  0.092
Method:                 Least Squares   F-statistic:                     3.300
Date:                Sat, 21 Nov 2020   Prob (F-statistic):             0.0257
Time:                        22:34:51   Log-Likelihood:                -118.84
No. Observations:                  69   AIC:                             245.7
Df Residuals:                      65   BIC:                             254.6
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.1085      0.182      0.598      0.552      -0.254       0.471
Mkt-RF         0.0888      0.042      2.126      0.037       0.005       0.172
SMB           -0.2053      0.074     -2.766      0.007      -0.353      -0.057
HML            0.0876      0.059      1.483      0.143      -0.030       0.206
==============================================================================
Omnibus:                        1.431   Durbin-Watson:                   1.790
Prob(Omnibus):                  0.489   Jarque-Bera (JB):                1.004
Skew:                           0.291   Prob(JB):                        0.605
Kurtosis:                       3.103   Cond. No.                         5.10
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### 2.4 An equivalent five factor model

In [88]:
#Equivalent five factore model
HMLexposure=GRS(FF5[0][['HML']],F5)
HMLexposure

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:27: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:30: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:31: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directl

(3.679635733342269,
 1,
 64,
 0.5850488213797067,
 0.32513183533504963,
 0.10571071034833784,
 0.2721599088071935,
 0.40138223105380066)

In [89]:
model = sm.OLS(FF5[0][['HML']], sm.add_constant(F5))

In [90]:
model.fit().summary() #Constant is around 0.0016, does not really add value or means that nothing is lost in explanation of average returns 
#if we drop HML from the model

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    HML   R-squared:                       0.437
Model:                            OLS   Adj. R-squared:                  0.401
Method:                 Least Squares   F-statistic:                     12.40
Date:                Sat, 21 Nov 2020   Prob (F-statistic):           1.59e-07
Time:                        22:34:51   Log-Likelihood:                -155.87
No. Observations:                  69   AIC:                             321.7
Df Residuals:                      64   BIC:                             332.9
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.5850      0.305     -1.917      0.060      -1.195       0.025
Mkt-RF         0.1315      0.078      1.688      0.096      -0.024       0.287
SMB            0.3701      0.127      2.907      0.005       0.116       0.624
RMW            0.1586      0.214      0.741      0.461      -0.269       0.586
CMA            1.0220      0.189      5.416      0.000       0.645       1.399
==============================================================================
Omnibus:                       14.068   Durbin-Watson:                   1.836
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               34.128
Skew:                          -0.513   Prob(JB):                     3.88e-08
Kurtosis:                       6.289   Cond. No.                         4.94
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [91]:
##Calculate HMLO
HMLO=model.fit().params[0]+model.fit().resid
HMLO

Date
2015-01   -1.345343
2015-02   -0.779544
2015-03   -0.902406
2015-04    3.357457
2015-05   -0.623699
             ...   
2020-05   -3.154896
2020-06   -3.613818
2020-07   -2.118145
2020-08   -2.811240
2020-09   -0.112501
Freq: M, Length: 69, dtype: float64

## 3. Market Beta

In [92]:
dt3=betaPortXs.mean()
dt3
dt3sd=betaPortXs.std()
dt3sd

SMALL LoBETA    5.186460
ME1 BETA2       5.715331
ME1 BETA3       5.849919
ME1 BETA4       6.910881
SMALL HiBETA    7.736210
ME2 BETA1       4.801250
ME2 BETA2       5.625603
ME2 BETA3       5.997465
ME2 BETA4       6.691243
ME2 BETA5       7.725870
ME3 BETA1       4.041038
ME3 BETA2       4.675053
ME3 BETA3       5.753545
ME3 BETA4       6.655152
ME3 BETA5       7.193159
ME4 BETA1       4.153232
ME4 BETA2       4.478599
ME4 BETA3       5.527862
ME4 BETA4       6.038418
ME4 BETA5       7.273820
BIG LoBETA      3.148733
ME5 BETA2       4.245207
ME5 BETA3       5.099482
ME5 BETA4       5.646492
BIG HiBETA      6.662548
dtype: float64

In [93]:

# List of tuples
students = [ (' ', '  ' , 'Mean' , ' ',' ',' ',' ',' ','SD',' ', ' '),
             (dt3[0], dt3[1], dt3[2] , dt3[3],dt3[4],' ',dt3sd[0], dt3sd[1], dt3sd[2] , dt3sd[3],dt3sd[4]) ,
             (dt3[5], dt3[6], dt3[7] , dt3[8],dt3[9],' ',dt3sd[5], dt3sd[6], dt3sd[7] , dt3sd[8],dt3sd[9]) ,
             (dt3[10], dt3[11], dt3[12] , dt3[13],dt3[14],' ',dt3sd[10], dt3sd[11], dt3sd[12] , dt3sd[13],dt3sd[14]) ,
             (dt3[15], dt3[16], dt3[17] , dt3[18],dt3[19],' ',dt3sd[15], dt3sd[16], dt3sd[17] , dt3sd[18],dt3sd[19]) ,
             (dt3[20], dt3[21], dt3[22] , dt3[23],dt3[24],' ',dt3sd[20], dt3sd[21], dt3sd[22] , dt3sd[23],dt3sd[24]) ]
# Create DataFrame object from a list of tuples
table3 = pd.DataFrame(students, columns = ['Low β' , '2', '3' , '4','High β',' ','Low β' , '2', '3' , '4','High β'], index=['','small', '2', '3' , '4' , 'Big'])
table3#Pnael a 

,Low β,2,3,4,High β,,Low β,2,3,4,High β
,,,Mean,,,,,,SD,,
small,0.621541,0.285294,0.261745,0.989841,0.171093,,5.18646,5.71533,5.84992,6.91088,7.73621
2,0.375451,0.372378,0.631281,0.65053,0.391293,,4.80125,5.6256,5.99747,6.69124,7.72587
3,0.360688,0.702675,0.528549,0.506012,0.499623,,4.04104,4.67505,5.75354,6.65515,7.19316
4,0.569372,0.972825,0.769025,0.790843,0.874168,,4.15323,4.4786,5.52786,6.03842,7.27382
Big,0.7558,0.969881,1.12024,1.08051,0.899416,,3.14873,4.24521,5.09948,5.64649,6.66255


In [94]:
#B/M=6/7 OP= 9 INV=10 PRIOR B= 8
dt3a=betaPort[6].mean()
dt3b=betaPort[9].mean()
# List of tuples
students = [ (' ', '  ' , 'B/M' , ' ',' ',' ',' ',' ','OP',' ', ' '),
             (dt3a[0], dt3a[1], dt3a[2] , dt3a[3],dt3a[4],' ',dt3b[0], dt3b[1], dt3b[2] , dt3b[3],dt3b[4]) ,
             (dt3a[5], dt3a[6], dt3a[7] , dt3a[8],dt3a[9],' ',dt3b[5], dt3b[6], dt3b[7] , dt3b[8],dt3b[9]) ,
             (dt3a[10], dt3a[11], dt3a[12] , dt3a[13],dt3a[14],' ',dt3b[10], dt3b[11], dt3b[12] , dt3b[13],dt3b[14]) ,
             (dt3a[15], dt3a[16], dt3a[17] , dt3a[18],dt3a[19],' ',dt3b[15],dt3b[16], dt3b[17] , dt3b[18],dt3b[19]) ,
             (dt3a[20], dt3a[21], dt3a[22] , dt3a[23],dt3a[24],' ',dt3b[20], dt3b[21], dt3b[22] , dt3b[23],dt3b[24]) ]
# Create DataFrame object from a list of tuples
table3a = pd.DataFrame(students, columns = ['Low β' , '2', '3' , '4','High β',' ','Low β' , '2', '3' , '4','High β'], index=['','small', '2', '3' , '4' , 'Big'])
table3a#Pnael a 

,Low β,2,3,4,High β,,Low β,2,3,4,High β
,,,B/M,,,,,,OP,,
small,0.665558,0.688307,0.676806,0.709241,0.642107,,0.0718449,0.103006,0.131572,-38.381,-0.336633
2,0.510325,0.525457,0.53358,0.507226,0.499422,,0.139023,0.254175,0.332122,0.211,0.131199
3,0.425455,0.42927,0.496312,0.472551,0.488897,,0.294009,0.31532,0.296823,0.246601,0.0505058
4,0.352417,0.329629,0.36252,0.444272,0.455317,,0.522438,0.419106,0.404232,0.693057,0.303364
Big,0.292801,0.293941,0.300478,0.483014,0.699048,,1.04661,0.817112,0.574913,0.88832,0.310894


In [95]:
#B/M=6/7 OP= 9 INV=10 PRIOR B= 8
dt3a=betaPort[10].mean()
dt3b=betaPort[8].mean()
# List of tuples
students = [ (' ', '  ' , 'Inv' , ' ',' ',' ',' ',' ','Prior-Beta',' ', ' '),
             (dt3a[0], dt3a[1], dt3a[2] , dt3a[3],dt3a[4],' ',dt3b[0], dt3b[1], dt3b[2] , dt3b[3],dt3b[4]) ,
             (dt3a[5], dt3a[6], dt3a[7] , dt3a[8],dt3a[9],' ',dt3b[5], dt3b[6], dt3b[7] , dt3b[8],dt3b[9]) ,
             (dt3a[10], dt3a[11], dt3a[12] , dt3a[13],dt3a[14],' ',dt3b[10], dt3b[11], dt3b[12] , dt3b[13],dt3b[14]) ,
             (dt3a[15], dt3a[16], dt3a[17] , dt3a[18],dt3a[19],' ',dt3b[15],dt3b[16], dt3b[17] , dt3b[18],dt3b[19]) ,
             (dt3a[20], dt3a[21], dt3a[22] , dt3a[23],dt3a[24],' ',dt3b[20], dt3b[21], dt3b[22] , dt3b[23],dt3b[24]) ]
# Create DataFrame object from a list of tuples
table3b = pd.DataFrame(students, columns = ['Low β' , '2', '3' , '4','High β',' ','Low β' , '2', '3' , '4','High β'], index=['','small', '2', '3' , '4' , 'Big'])
table3b#Pnael a 

,Low β,2,3,4,High β,,Low β,2,3,4,High β
,,,Inv,,,,,,Prior-Beta,,
small,0.0764942,0.0828029,0.0750116,0.0753884,0.0802101,,0.088113,0.825767,1.21671,1.63378,2.92458
2,0.0961841,0.0926058,0.0972551,0.100196,0.136067,,0.182239,0.842355,1.21005,1.63081,2.81294
3,0.11191,0.0852246,0.0969304,0.0929725,0.145183,,0.204454,0.842684,1.22875,1.6454,2.58728
4,0.0848739,0.105713,0.091971,0.10233,0.131126,,0.267251,0.83478,1.22259,1.62967,2.43043
Big,0.0717841,0.0862652,0.0816029,0.0659971,0.0515333,,0.30393,0.846981,1.20876,1.64388,2.2184


In [96]:
##Continue to the deeper regression on each port
def regMarket(R,F):
    alpha=[]
    beta=[]
    tstata=[]
    tstatb=[]
    c=[]
    tstatc=[]
    d=[]
    tstatd=[]
    
    for p in R.columns:
        model = sm.OLS(R[:][p], sm.add_constant(F[:]))
        fit = model.fit()
        alpha.append(fit.params["const"])
        beta.append(fit.params[1])
        tstata.append(fit.tvalues[0])
        tstatb.append(fit.tvalues[1])
    
        
    return (alpha,beta,tstata,tstatb)
def regCAPM(R,F):
    alpha=[]
    beta=[]
    tstata=[]
    tstatb=[]
    c=[]
    tstatc=[]
    d=[]
    tstatd=[]
    
    for p in R.columns:
        model = sm.OLS(R[:][p], sm.add_constant(F[:]))
        fit = model.fit()
        alpha.append(fit.params["const"])
        beta.append(fit.params[1])
        tstata.append(fit.tvalues[0])
        tstatb.append(fit.tvalues[1])
        c.append(fit.params[2])
        tstatc.append(fit.tvalues[2])
        d.append(fit.params[3])
        tstatd.append(fit.tvalues[3])
        
    return (alpha,beta,tstata,tstatb,c,tstatc,d,tstatd)
def regFourFactor(R,F):
    alpha=[]
    beta=[]
    tstata=[]
    tstatb=[]
    c=[]
    tstatc=[]
    d=[]
    tstatd=[]
    e=[]
    tstate=[]
    for p in R.columns:
        model = sm.OLS(R[:][p], sm.add_constant(F[:]))
        fit = model.fit()
        alpha.append(fit.params["const"])
        beta.append(fit.params[1])
        tstata.append(fit.tvalues[0])
        tstatb.append(fit.tvalues[1])
        c.append(fit.params[2])
        tstatc.append(fit.tvalues[2])
        d.append(fit.params[3])
        tstatd.append(fit.params[3])
        e.append(fit.params[4])
        tstate.append(fit.tvalues[4])
    return (alpha,beta,tstata,tstatb,c,tstatc,d,tstatd,e,tstate)
            
def regFivefactor(R,F):
    alpha=[]
    beta=[]
    s=[]
    h=[]
    r=[]
    c=[]
    tstata=[]
    tstatb=[]
    tstats=[]
    tstath=[]
    tstatr=[]
    tstatc=[]
       
    for p in R.columns:
        model = sm.OLS(R[:][p], sm.add_constant(F[:]))
        fit = model.fit()
        alpha.append(fit.params["const"])
        beta.append(fit.params[1])
        s.append(fit.params[2])
        h.append(fit.params[3])
        r.append(fit.params[4])
        c.append(fit.params[5])
        tstata.append(fit.tvalues[0])
        tstatb.append(fit.tvalues[1])
        tstats.append(fit.tvalues[2])
        tstath.append(fit.tvalues[3])
        tstatr.append(fit.tvalues[4])
        tstatc.append(fit.tvalues[5])
    return (alpha,beta,tstata,tstatb,s,tstats,h,tstath,r,tstatr,c,tstatc)
def regSixfactor(R,F):
    alpha=[]
    beta=[]
    s=[]
    h=[]
    r=[]
    c=[]
    tstata=[]
    tstatb=[]
    tstats=[]
    tstath=[]
    tstatr=[]
    tstatc=[]
    m=[]
    tstatm=[]
       
    for p in R.columns:
        model = sm.OLS(R[:][p], sm.add_constant(F[:]))
        fit = model.fit()
        alpha.append(fit.params["const"])
        beta.append(fit.params[1])
        s.append(fit.params[2])
        h.append(fit.params[3])
        r.append(fit.params[4])
        c.append(fit.params[5])
        m.append(fit.params[6])
        tstata.append(fit.tvalues[0])
        tstatb.append(fit.tvalues[1])
        tstats.append(fit.tvalues[2])
        tstath.append(fit.tvalues[3])
        tstatr.append(fit.tvalues[4])
        tstatc.append(fit.tvalues[5])
        tstatm.append(fit.tvalues[6])
    return (alpha,beta,tstata,tstatb,s,tstats,h,tstath,r,tstatr,c,tstatc,m,tstatm)

In [97]:

result3a=regMarket(betaPortXs,F1)


In [98]:

# List of tuples
regcapmdata = [ ('', '  ' , 'a' , ' ',' ',' ',' ',' ','t(a)',' ', ' '),
             (result3a[0][0], result3a[0][1], result3a[0][2] , result3a[0][3],result3a[0][4],' ',result3a[2][0], result3a[2][1], result3a[2][2] , result3a[2][3],result3a[2][4]) ,
             (result3a[0][5], result3a[0][6], result3a[0][7] , result3a[0][8],result3a[0][9],' ',result3a[2][5], result3a[2][6], result3a[2][7] , result3a[2][8],result3a[2][9]) ,
             (result3a[0][10], result3a[0][11], result3a[0][12] , result3a[0][13],result3a[0][14],' ',result3a[2][10], result3a[2][11], result3a[2][12] , result3a[2][13],result3a[2][14]) ,
             (result3a[0][15], result3a[0][16], result3a[0][17] , result3a[0][18],result3a[0][19],' ',result3a[2][15], result3a[2][16], result3a[2][17] , result3a[2][18],result3a[2][19]) ,
             (result3a[0][20], result3a[0][21], result3a[0][22] , result3a[0][23],result3a[0][24],' ',result3a[2][20], result3a[2][21], result3a[2][22] , result3a[2][23],result3a[2][24]),
             ('', '  ' , 'b' , ' ',' ',' ',' ',' ','t(b)',' ', ' '),
             (result3a[1][0], result3a[1][1], result3a[1][2] , result3a[1][3],result3a[1][4],' ',result3a[3][0], result3a[3][1], result3a[3][2] , result3a[3][3],result3a[3][4]) ,
             (result3a[1][5], result3a[1][6], result3a[1][7] , result3a[1][8],result3a[1][9],' ',result3a[3][5], result3a[3][6], result3a[3][7] , result3a[3][8],result3a[3][9]) ,
             (result3a[1][10], result3a[1][11], result3a[1][12] , result3a[1][13],result3a[1][14],' ',result3a[3][10], result3a[3][11], result3a[3][12] , result3a[3][13],result3a[3][14]) ,
             (result3a[1][15], result3a[1][16], result3a[1][17] , result3a[1][18],result3a[1][19],' ',result3a[3][15], result3a[3][16], result3a[3][17] , result3a[3][18],result3a[3][19]) ,
             (result3a[1][20], result3a[1][21], result3a[1][22] , result3a[1][23],result3a[1][24],' ',result3a[3][20], result3a[3][21], result3a[3][22] , result3a[3][23],result3a[3][24])]
# Create DataFrame object from a list of tuples
table4a = pd.DataFrame(regcapmdata, columns = ['Low β' , '2', '3' , '4','High β',' ','Low β' , '2', '3' , '4','High β'], index=['','small', '2', '3' , '4' , 'Big','','small', '2', '3' , '4' , 'Big'])
table4a

,Low β,2,3,4,High β,,Low β,2,3,4,High β
,,,a,,,,,,t(a),,
small,-0.269191,-0.672282,-0.762932,-0.227329,-1.20158,,-0.746712,-1.6098,-1.96426,-0.502355,-2.41908
2,-0.487488,-0.654547,-0.491065,-0.611274,-1.03502,,-1.64069,-1.97626,-1.53083,-1.76862,-2.37005
3,-0.383772,-0.189596,-0.577555,-0.768848,-0.851237,,-1.66676,-0.834968,-2.15528,-2.42831,-2.24753
4,-0.198367,0.0808206,-0.333104,-0.40194,-0.527772,,-0.849148,0.520344,-1.75915,-1.75747,-1.58262
Big,0.185552,0.109683,0.116689,-0.0221655,-0.344197,,0.974981,0.971534,0.58344,-0.0938558,-0.955268
,,,b,,,,,,t(b),,
small,0.972783,1.04579,1.11907,1.32929,1.49912,,12.0784,11.2089,12.8965,13.1485,13.5094
2,0.942431,1.12152,1.22573,1.37804,1.5577,,14.1976,15.157,17.1036,17.8469,15.966
3,0.813038,0.974465,1.208,1.3923,1.4753,,15.8057,19.2092,20.178,19.6833,17.4356


In [99]:
## Five Factor regression with HML->HMLO
test=F2.copy()
test=test.drop(['HML'],axis=1)
test['HMLO']=HMLO
test['RMW']=F5['RMW']
test['CMA']=F5['CMA']

In [100]:
result3a=regFivefactor(betaPortXs,test) #For five factor regression

In [101]:
#List of tuples
reg5Fdata = [ ('', '  ' , 'a' , ' ',' ',' ',' ',' ','t(a)',' ', ' '),
             (result3a[0][0], result3a[0][1], result3a[0][2] , result3a[0][3],result3a[0][4],' ',result3a[2][0], result3a[2][1], result3a[2][2] , result3a[2][3],result3a[2][4]) ,
             (result3a[0][5], result3a[0][6], result3a[0][7] , result3a[0][8],result3a[0][9],' ',result3a[2][5], result3a[2][6], result3a[2][7] , result3a[2][8],result3a[2][9]) ,
             (result3a[0][10], result3a[0][11], result3a[0][12] , result3a[0][13],result3a[0][14],' ',result3a[2][10], result3a[2][11], result3a[2][12] , result3a[2][13],result3a[2][14]) ,
             (result3a[0][15], result3a[0][16], result3a[0][17] , result3a[0][18],result3a[0][19],' ',result3a[2][15], result3a[2][16], result3a[2][17] , result3a[2][18],result3a[2][19]) ,
             (result3a[0][20], result3a[0][21], result3a[0][22] , result3a[0][23],result3a[0][24],' ',result3a[2][20], result3a[2][21], result3a[2][22] , result3a[2][23],result3a[2][24]),
             ('', '  ' , 'b' , ' ',' ',' ',' ',' ','t(b)',' ', ' '),
             (result3a[1][0], result3a[1][1], result3a[1][2] , result3a[1][3],result3a[1][4],' ',result3a[3][0], result3a[3][1], result3a[3][2] , result3a[3][3],result3a[3][4]) ,
             (result3a[1][5], result3a[1][6], result3a[1][7] , result3a[1][8],result3a[1][9],' ',result3a[3][5], result3a[3][6], result3a[3][7] , result3a[3][8],result3a[3][9]) ,
             (result3a[1][10], result3a[1][11], result3a[1][12] , result3a[1][13],result3a[1][14],' ',result3a[3][10], result3a[3][11], result3a[3][12] , result3a[3][13],result3a[3][14]) ,
             (result3a[1][15], result3a[1][16], result3a[1][17] , result3a[1][18],result3a[1][19],' ',result3a[3][15], result3a[3][16], result3a[3][17] , result3a[3][18],result3a[3][19]) ,
             (result3a[1][20], result3a[1][21], result3a[1][22] , result3a[1][23],result3a[1][24],' ',result3a[3][20], result3a[3][21], result3a[3][22] , result3a[3][23],result3a[3][24]),
             ('', '  ' , 's' , ' ',' ',' ',' ',' ','t(s)',' ', ' '),
             (result3a[4][0], result3a[4][1], result3a[4][2] , result3a[4][3],result3a[4][4],' ',result3a[5][0], result3a[5][1], result3a[5][2] , result3a[5][3],result3a[5][4]),
             (result3a[4][5], result3a[4][6], result3a[4][7] , result3a[4][8],result3a[4][9],' ',result3a[5][5], result3a[5][6], result3a[5][7] , result3a[5][8],result3a[5][9]),
             (result3a[4][10], result3a[4][11], result3a[4][12] , result3a[4][13],result3a[4][14],' ',result3a[5][10], result3a[5][11], result3a[5][12] , result3a[5][13],result3a[5][14]),
             (result3a[4][15], result3a[4][16], result3a[4][17] , result3a[4][18],result3a[4][19],' ',result3a[5][15], result3a[5][16], result3a[5][17] , result3a[5][18],result3a[5][19]),
             (result3a[4][20], result3a[4][21], result3a[4][22] , result3a[4][23],result3a[4][24],' ',result3a[5][20], result3a[5][21], result3a[5][22] , result3a[5][23],result3a[5][24]),
             ('', '  ' , 'h' , ' ',' ',' ',' ',' ','t(h)',' ', ' '),
             (result3a[6][0], result3a[6][1], result3a[6][2] , result3a[6][3],result3a[6][4],' ',result3a[7][0], result3a[7][1], result3a[7][2] , result3a[7][3],result3a[7][4]) ,
             (result3a[6][5], result3a[6][6], result3a[6][7] , result3a[6][8],result3a[6][9],' ',result3a[7][5], result3a[7][6], result3a[7][7] , result3a[7][8],result3a[7][9]) ,
             (result3a[6][10], result3a[6][11], result3a[6][12] , result3a[6][13],result3a[6][14],' ',result3a[7][10], result3a[7][11], result3a[7][12] , result3a[7][13],result3a[7][14]) ,
             (result3a[6][15], result3a[6][16], result3a[6][17] , result3a[6][18],result3a[6][19],' ',result3a[7][15], result3a[7][16], result3a[7][17] , result3a[7][18],result3a[7][19]) ,
             (result3a[6][20], result3a[6][21], result3a[6][22] , result3a[6][23],result3a[6][24],' ',result3a[7][20], result3a[7][21], result3a[7][22] , result3a[7][23],result3a[7][24]), 
             ('', '  ' , 'r' , ' ',' ',' ',' ',' ','t(r)',' ', ' '),
             (result3a[8][0], result3a[8][1], result3a[8][2] , result3a[8][3],result3a[8][4],' ',result3a[9][0], result3a[9][1], result3a[9][2] , result3a[9][3],result3a[9][4]) ,
             (result3a[8][5], result3a[8][6], result3a[8][7] , result3a[8][8],result3a[8][9],' ',result3a[9][5], result3a[9][6], result3a[9][7] , result3a[9][8],result3a[9][9]) ,
             (result3a[8][10], result3a[8][11], result3a[8][12] , result3a[8][13],result3a[8][14],' ',result3a[9][10], result3a[9][11], result3a[9][12] , result3a[9][13],result3a[9][14]) ,
             (result3a[8][15], result3a[8][16], result3a[8][17] , result3a[8][18],result3a[8][19],' ',result3a[9][15], result3a[9][16], result3a[9][17] , result3a[9][18],result3a[9][19]) ,
             (result3a[8][20], result3a[8][21], result3a[8][22] , result3a[8][23],result3a[8][24],' ',result3a[9][20], result3a[9][21], result3a[9][22] , result3a[9][23],result3a[9][24]),
             ('', '  ' , 'c' , ' ',' ',' ',' ',' ','t(c)',' ', ' '),
             (result3a[10][0], result3a[10][1], result3a[10][2] , result3a[10][3],result3a[10][4],' ',result3a[11][0], result3a[11][1], result3a[11][2] , result3a[11][3],result3a[11][4]) ,
             (result3a[10][5], result3a[10][6], result3a[10][7] , result3a[10][8],result3a[10][9],' ',result3a[11][5], result3a[11][6], result3a[11][7] , result3a[11][8],result3a[11][9]) ,
             (result3a[10][10], result3a[10][11], result3a[10][12] , result3a[10][13],result3a[10][14],' ',result3a[11][10], result3a[11][11], result3a[11][12] , result3a[11][13],result3a[11][14]) ,
             (result3a[10][15], result3a[10][16], result3a[10][17] , result3a[10][18],result3a[10][19],' ',result3a[11][15], result3a[11][16], result3a[11][17] , result3a[11][18],result3a[11][19]) ,
             (result3a[10][20], result3a[10][21], result3a[10][22] , result3a[10][23],result3a[10][24],' ',result3a[11][20], result3a[11][21], result3a[11][22] , result3a[11][23],result3a[11][24])]



# Create DataFrame object from a list of tuples
table4b = pd.DataFrame(reg5Fdata, columns = ['Low β' , '2', '3' , '4','High β',' ','Low β' , '2', '3' , '4','High β'], index=['','small', '2', '3' , '4' , 'Big','','small', '2', '3' , '4' , 'Big','','small', '2', '3' , '4' , 'Big','','small', '2', '3' , '4' , 'Big','','small', '2', '3' , '4' , 'Big','','small', '2', '3' , '4' , 'Big'])
table4b

,Low β,2,3,4,High β,,Low β,2,3,4,High β
,,,a,,,,,,t(a),,
small,0.414826,0.245594,0.0184353,0.503954,-0.461744,,2.34575,1.53056,0.10527,2.13408,-1.60709
2,0.027184,0.0697798,0.171065,0.0806783,-0.423674,,0.136039,0.429123,1.2223,0.555428,-2.079
3,-0.205219,0.145518,-0.0141328,-0.106927,-0.319931,,-0.911562,0.813585,-0.0988628,-0.66968,-1.23585
4,-0.092345,0.178462,-0.10348,0.0331176,-0.0786136,,-0.37895,1.14878,-0.618241,0.20647,-0.297644
Big,-0.035499,0.0225234,-0.0207419,0.217985,0.311504,,-0.213306,0.208133,-0.099664,0.962784,1.08357
,,,b,,,,,,t(b),,
small,0.729259,0.765319,0.868014,1.03873,1.23732,,16.6116,19.2127,19.9661,17.7188,17.3474
2,0.79789,0.923734,1.00499,1.15299,1.33789,,16.0845,22.883,28.9262,31.9749,26.4459
3,0.716811,0.842459,1.06803,1.18792,1.27314,,12.8259,18.9736,30.0956,29.9698,19.8106


### 4.NI Net Share Issues

In [102]:
dt3=niPortXs.mean()
dt3
dt3sd=niPortXs.std()
dt3sd.head()

SMALL NegNI     6.079616
SMALL ZeroNI    5.906324
SMALL LoNI      5.743685
ME1 NI2         6.342249
ME1 NI3         6.157413
dtype: float64

In [103]:
# List of tuples
students = [ (' ',' ',' ','Mean',' ',' ',' ',' ',' ',' ',' ','SD',' ',' ', ' '),
             (dt3[0], dt3[1], dt3[2] , dt3[3],dt3[4],dt3[5], dt3[6],' ',dt3sd[0], dt3sd[1], dt3sd[2] , dt3sd[3],dt3sd[4],dt3sd[5], dt3sd[6]),
             (dt3[7], dt3[8], dt3[9] , dt3[10],dt3[11],dt3[12], dt3[13],' ',dt3sd[7], dt3sd[8], dt3sd[9] , dt3sd[10],dt3sd[11],dt3sd[12], dt3sd[13]),
             (dt3[14], dt3[15], dt3[16],dt3[17],dt3[18],dt3[19], dt3[20],' ',dt3sd[14], dt3sd[15], dt3sd[16] , dt3sd[17],dt3sd[18],dt3sd[19], dt3sd[20]),
             (dt3[21], dt3[22], dt3[23] , dt3[24],dt3[25],dt3[26], dt3[27],' ',dt3sd[21], dt3sd[22], dt3sd[23] , dt3sd[24],dt3sd[25],dt3sd[26], dt3sd[27]),
             (dt3[28], dt3[29], dt3[30] , dt3[31],dt3[32],dt3[33], dt3[34],' ',dt3sd[28], dt3sd[29], dt3sd[30] , dt3sd[31],dt3sd[32],dt3sd[33], dt3sd[34])]
# Create DataFrame object from a list of tuples
table3 = pd.DataFrame(students, columns = ['Neg','Zero','Low' , '2', '3' , '4','High ',' ','Neg','Zero','Low', '2', '3' , '4','High'], index=['','small', '2', '3' , '4' , 'Big'])
table3#Pnael a 

,Neg,Zero,Low,2,3,4,High,,Neg,Zero,Low,2,3,4,High
,,,,Mean,,,,,,,,SD,,,
small,0.28389,0.559274,0.434716,0.334739,0.559238,0.822767,0.477219,,6.07962,5.90632,5.74369,6.34225,6.15741,6.33876,7.09326
2,0.363172,0.741562,0.277186,0.0709014,0.509755,1.11935,0.754226,,6.23974,7.07581,5.54003,5.9654,5.70884,6.084,7.4878
3,0.583597,0.26387,0.177683,0.494364,0.541719,0.735367,1.06668,,5.73365,5.93905,5.03107,6.10416,5.38881,5.6598,6.57914
4,0.801149,1.026,0.822797,0.423994,0.75311,1.33388,0.594835,,5.20764,4.53586,5.19867,5.60693,5.44146,5.42764,6.20486
Big,0.928932,1.07089,0.662845,0.821774,1.8106,1.4601,0.48792,,4.26488,4.75165,4.09224,4.29992,5.80823,5.5271,5.27913


In [104]:
#6: B/M  10:OP 11:inv  8:NI
dt3a=niPort[6].mean()
dt3b=niPort[10].mean()
dt3b['BIG ZeroNI']=niPort[10]['BIG ZeroNI'].median()#Some error in the dataset. Already screenshot
dt3b.round(6)
# List of tuples
students = [ (' ',' ',' ','B/M',' ',' ',' ',' ',' ',' ',' ','OP',' ',' ', ' '),
             (dt3a[0], dt3a[1], dt3a[2] , dt3a[3],dt3a[4],dt3a[5], dt3a[6],' ',dt3b[0], dt3b[1], dt3b[2] , dt3b[3],dt3b[4],dt3b[5], dt3b[6]),
             (dt3a[7], dt3a[8], dt3a[9] , dt3a[10],dt3a[11],dt3a[12], dt3a[13],' ',dt3b[7], dt3b[8], dt3b[9] , dt3b[10],dt3b[11],dt3b[12], dt3b[13]),
             (dt3a[14], dt3a[15], dt3a[16],dt3a[17],dt3a[18],dt3a[19], dt3a[20],' ',dt3b[14], dt3b[15], dt3b[16] , dt3b[17],dt3b[18],dt3b[19], dt3b[20]),
             (dt3a[21], dt3a[22], dt3a[23] , dt3a[24],dt3a[25],dt3a[26], dt3a[27],' ',dt3b[21], dt3b[22], dt3b[23] , dt3b[24],dt3b[25],dt3b[26], dt3b[27]),
             (dt3a[28], dt3a[29], dt3a[30] , dt3a[31],dt3a[32],dt3a[33], dt3a[34],' ',dt3b[28], dt3b[29], dt3b[30] , dt3b[31],dt3b[32],dt3b[33], dt3b[34])]
# Create DataFrame object from a list of tuples
table3a = pd.DataFrame(students, columns = ['Neg','Zero','Low' , '2', '3' , '4','High ',' ','Neg','Zero','Low', '2', '3' , '4','High'], index=['','small', '2', '3' , '4' , 'Big'])
table3a

,Neg,Zero,Low,2,3,4,High,,Neg,Zero,Low,2,3,4,High
,,,,B/M,,,,,,,,OP,,,
small,0.768258,0.799332,0.777141,0.717741,0.684058,0.530348,0.53998,,0.19342,0.564543,-0.0712536,0.112907,0.166829,-0.0493942,-17.1116
2,0.54967,0.677909,0.64598,0.53638,0.493257,0.388487,0.434625,,0.371171,0.313175,0.369359,0.27562,0.296286,0.18122,-0.227335
3,0.473023,0.52787,0.601987,0.470577,0.4253,0.332036,0.455246,,0.475171,0.173913,0.318622,0.268606,0.211155,0.0343406,-0.121138
4,0.391032,0.377243,0.429548,0.443912,0.325287,0.271712,0.422203,,0.617262,0.357567,0.71887,0.3461,0.532594,0.202003,0.0281797
Big,0.3252,0.483622,0.432381,0.341325,0.204387,0.335636,0.403207,,1.08638,0.3083,0.635251,0.228864,0.386562,0.306691,0.259412


In [105]:
#6: B/M  10:OP 11:inv  8:NI
dt3a=niPort[11].mean()
dt3b=niPort[8].mean()
# List of tuples
students = [ (' ',' ',' ','Inv',' ',' ',' ',' ',' ',' ',' ','NI',' ',' ', ' '),
             (dt3a[0], dt3a[1], dt3a[2] , dt3a[3],dt3a[4],dt3a[5], dt3a[6],' ',dt3b[0], dt3b[1], dt3b[2] , dt3b[3],dt3b[4],dt3b[5], dt3b[6]),
             (dt3a[7], dt3a[8], dt3a[9] , dt3a[10],dt3a[11],dt3a[12], dt3a[13],' ',dt3b[7], dt3b[8], dt3b[9] , dt3b[10],dt3b[11],dt3b[12], dt3b[13]),
             (dt3a[14], dt3a[15], dt3a[16],dt3a[17],dt3a[18],dt3a[19], dt3a[20],' ',dt3b[14], dt3b[15], dt3b[16] , dt3b[17],dt3b[18],dt3b[19], dt3b[20]),
             (dt3a[21], dt3a[22], dt3a[23] , dt3a[24],dt3a[25],dt3a[26], dt3a[27],' ',dt3b[21], dt3b[22], dt3b[23] , dt3b[24],dt3b[25],dt3b[26], dt3b[27]),
             (dt3a[28], dt3a[29], dt3a[30] , dt3a[31],dt3a[32],dt3a[33], dt3a[34],' ',dt3b[28], dt3b[29], dt3b[30] , dt3b[31],dt3b[32],dt3b[33], dt3b[34])]
# Create DataFrame object from a list of tuples
table3b = pd.DataFrame(students, columns = ['Neg','Zero','Low' , '2', '3' , '4','High ',' ','Neg','Zero','Low', '2', '3' , '4','High'], index=['','small', '2', '3' , '4' , 'Big'])
table3b

,Neg,Zero,Low,2,3,4,High,,Neg,Zero,Low,2,3,4,High
,,,,Inv,,,,,,,,NI,,,
small,0.0293058,0.109287,0.000623188,0.0321522,0.0274348,0.0660333,0.475613,,-7.66856,0,0.187055,0.557646,1.14925,3.29119,59.1345
2,0.0459623,0.0355058,0.0482696,0.057471,0.0709246,0.133751,0.450699,,-5.16644,0,0.201159,0.548609,1.16164,3.30093,50.3632
3,0.0342652,0.071758,0.0445739,0.0666899,0.0943667,0.140339,0.385438,,-3.69655,0,0.188641,0.551371,1.14008,3.38429,36.3536
4,0.038,0.00121884,0.0654855,0.0748449,0.0902841,0.176094,0.446223,,-3.62177,0,0.186325,0.525016,1.18299,3.43888,31.5827
Big,0.0396826,0.0767725,0.0589957,0.0821899,0.191778,0.175855,0.486551,,-3.09455,0,0.1518,0.562404,1.24074,3.23573,27.6391


### Regression

In [106]:
result3a=regCAPM(niPortXs,F2)
dt3a=result3a[0]
dt3b=result3a[2]

In [107]:
# List of tuples
regcapmdata = [ (' ',' ',' ','a',' ',' ',' ',' ',' ',' ',' ','t(a)',' ',' ', ' '),
             (dt3a[0], dt3a[1], dt3a[2] , dt3a[3],dt3a[4],dt3a[5], dt3a[6],' ',dt3b[0], dt3b[1], dt3b[2] , dt3b[3],dt3b[4],dt3b[5], dt3b[6]),
             (dt3a[7], dt3a[8], dt3a[9] , dt3a[10],dt3a[11],dt3a[12], dt3a[13],' ',dt3b[7], dt3b[8], dt3b[9] , dt3b[10],dt3b[11],dt3b[12], dt3b[13]),
             (dt3a[14], dt3a[15], dt3a[16],dt3a[17],dt3a[18],dt3a[19], dt3a[20],' ',dt3b[14], dt3b[15], dt3b[16] , dt3b[17],dt3b[18],dt3b[19], dt3b[20]),
             (dt3a[21], dt3a[22], dt3a[23] , dt3a[24],dt3a[25],dt3a[26], dt3a[27],' ',dt3b[21], dt3b[22], dt3b[23] , dt3b[24],dt3b[25],dt3b[26], dt3b[27]),
             (dt3a[28], dt3a[29], dt3a[30] , dt3a[31],dt3a[32],dt3a[33], dt3a[34],' ',dt3b[28], dt3b[29], dt3b[30] , dt3b[31],dt3b[32],dt3b[33], dt3b[34])]
# Create DataFrame object from a list of tuples
table4a = pd.DataFrame(regcapmdata, columns = ['Neg','Zero','Low' , '2', '3' , '4','High ',' ','Neg','Zero','Low', '2', '3' , '4','High'], index=['','small', '2', '3' , '4' , 'Big'])
table4a

,Neg,Zero,Low,2,3,4,High,,Neg,Zero,Low,2,3,4,High
,,,,a,,,,,,,,t(a),,,
small,0.210438,0.471501,0.432889,0.142545,0.268947,0.0706309,-0.322108,,1.27069,0.990206,2.04386,0.774289,1.31344,0.348827,-0.954717
2,-0.040922,0.638543,0.0517753,-0.267551,-0.0472492,0.386993,-0.263994,,-0.275427,1.06104,0.272594,-1.39153,-0.228803,1.70204,-0.935767
3,0.100404,-0.204502,-0.24306,-0.0525854,-0.111625,-0.336955,0.047997,,0.761685,-0.412769,-1.27787,-0.232314,-0.571141,-1.52652,0.182271
4,0.133201,0.203557,-0.144619,-0.249401,-0.183312,0.067825,-0.515239,,1.01633,0.518397,-0.663735,-1.17479,-0.891837,0.263546,-1.64192
Big,0.0529059,0.44505,-0.0574264,-0.120177,0.123142,0.0500402,-0.530327,,0.791123,0.928131,-0.319919,-0.46342,0.325382,0.151031,-1.73832


In [108]:
result3a=regFivefactor(niPortXs,test)  #test already include HMLO
dt3a=result3a[0] 
dt3b=result3a[2]
dt3c=result3a[6] 

dt3d=result3a[7] 
dt3e=result3a[8]
dt3f=result3a[9]
dt3g=result3a[10]
dt3h=result3a[11]

In [109]:
reg5FdataNI = [ (' ',' ',' ','a',' ',' ',' ',' ',' ',' ',' ','t(a)',' ',' ', ' '),
             (dt3a[0], dt3a[1], dt3a[2] , dt3a[3],dt3a[4],dt3a[5], dt3a[6],' ',dt3b[0], dt3b[1], dt3b[2] , dt3b[3],dt3b[4],dt3b[5], dt3b[6]),
             (dt3a[7], dt3a[8], dt3a[9] , dt3a[10],dt3a[11],dt3a[12], dt3a[13],' ',dt3b[7], dt3b[8], dt3b[9] , dt3b[10],dt3b[11],dt3b[12], dt3b[13]),
             (dt3a[14], dt3a[15], dt3a[16],dt3a[17],dt3a[18],dt3a[19], dt3a[20],' ',dt3b[14], dt3b[15], dt3b[16] , dt3b[17],dt3b[18],dt3b[19], dt3b[20]),
             (dt3a[21], dt3a[22], dt3a[23] , dt3a[24],dt3a[25],dt3a[26], dt3a[27],' ',dt3b[21], dt3b[22], dt3b[23] , dt3b[24],dt3b[25],dt3b[26], dt3b[27]),
             (dt3a[28], dt3a[29], dt3a[30] , dt3a[31],dt3a[32],dt3a[33], dt3a[34],' ',dt3b[28], dt3b[29], dt3b[30] , dt3b[31],dt3b[32],dt3b[33], dt3b[34]),
             (' ',' ',' ','h',' ',' ',' ',' ',' ',' ',' ','t(h)',' ',' ', ' '),
             (dt3c[0], dt3c[1], dt3c[2] , dt3c[3],dt3c[4],dt3c[5], dt3c[6],' ',dt3d[0], dt3d[1], dt3d[2] , dt3d[3],dt3d[4],dt3d[5], dt3d[6]),
             (dt3c[7], dt3c[8], dt3c[9] , dt3c[10],dt3c[11],dt3c[12], dt3c[13],' ',dt3d[7], dt3d[8], dt3d[9] , dt3d[10],dt3d[11],dt3d[12], dt3d[13]),
             (dt3c[14], dt3c[15], dt3c[16],dt3c[17],dt3c[18],dt3c[19], dt3c[20],' ',dt3d[14], dt3d[15], dt3d[16] , dt3d[17],dt3d[18],dt3d[19], dt3d[20]),
             (dt3c[21], dt3c[22], dt3c[23] , dt3c[24],dt3c[25],dt3c[26], dt3c[27],' ',dt3d[21], dt3d[22], dt3d[23] , dt3d[24],dt3d[25],dt3d[26], dt3d[27]),
             (dt3c[28], dt3c[29], dt3c[30] , dt3c[31],dt3c[32],dt3c[33], dt3c[34],' ',dt3d[28], dt3d[29], dt3d[30] , dt3d[31],dt3d[32],dt3d[33], dt3d[34]), 
             (' ',' ',' ','r',' ',' ',' ',' ',' ',' ',' ','t(r)',' ',' ', ' '),
             (dt3e[0], dt3e[1], dt3e[2] , dt3e[3],dt3e[4],dt3e[5], dt3e[6],' ',dt3f[0], dt3f[1], dt3f[2] , dt3f[3],dt3f[4],dt3f[5], dt3f[6]),
             (dt3e[7], dt3e[8], dt3e[9] , dt3e[10],dt3e[11],dt3e[12], dt3e[13],' ',dt3f[7], dt3f[8], dt3f[9] , dt3f[10],dt3f[11],dt3f[12], dt3f[13]),
             (dt3e[14], dt3e[15], dt3e[16],dt3e[17],dt3e[18],dt3e[19], dt3e[20],' ',dt3f[14], dt3f[15], dt3f[16] , dt3f[17],dt3f[18],dt3f[19], dt3f[20]),
             (dt3e[21], dt3e[22], dt3e[23] , dt3e[24],dt3e[25],dt3e[26], dt3e[27],' ',dt3f[21], dt3f[22], dt3f[23] , dt3f[24],dt3f[25],dt3f[26], dt3f[27]),
             (dt3e[28], dt3e[29], dt3e[30] , dt3e[31],dt3e[32],dt3e[33], dt3e[34],' ',dt3f[28], dt3f[29], dt3f[30] , dt3f[31],dt3f[32],dt3f[33], dt3f[34]),
             (' ',' ',' ','c',' ',' ',' ',' ',' ',' ',' ','t(c)',' ',' ', ' '),
             (dt3g[0], dt3g[1], dt3g[2] , dt3g[3],dt3g[4],dt3g[5], dt3g[6],' ',dt3h[0], dt3h[1], dt3h[2] , dt3h[3],dt3h[4],dt3h[5], dt3h[6]),
             (dt3g[7], dt3g[8], dt3g[9] , dt3g[10],dt3g[11],dt3g[12], dt3g[13],' ',dt3h[7], dt3h[8], dt3h[9] , dt3h[10],dt3h[11],dt3h[12], dt3h[13]),
             (dt3g[14], dt3g[15], dt3g[16],dt3g[17],dt3g[18],dt3g[19], dt3g[20],' ',dt3h[14], dt3h[15], dt3h[16] , dt3h[17],dt3h[18],dt3h[19], dt3h[20]),
             (dt3g[21], dt3g[22], dt3g[23] , dt3g[24],dt3g[25],dt3g[26], dt3g[27],' ',dt3h[21], dt3h[22], dt3h[23] , dt3h[24],dt3h[25],dt3h[26], dt3h[27]),
             (dt3g[28], dt3g[29], dt3g[30] , dt3g[31],dt3g[32],dt3g[33], dt3g[34],' ',dt3h[28], dt3h[29], dt3h[30] , dt3h[31],dt3h[32],dt3h[33], dt3h[34])]


# Create DataFrame object from a list of tuples
table4b = pd.DataFrame(reg5FdataNI, columns =['Neg','Zero','Low' , '2', '3' , '4','High ',' ','Neg','Zero','Low', '2', '3' , '4','High'], index=['','small', '2', '3' , '4' , 'Big','','small', '2', '3' , '4' , 'Big','','small', '2', '3' , '4' , 'Big','','small', '2', '3' , '4' , 'Big'])
table4b

,Neg,Zero,Low,2,3,4,High,,Neg,Zero,Low,2,3,4,High
,,,,a,,,,,,,,t(a),,,
small,0.190763,0.429379,0.400594,0.143169,0.268885,0.0721164,-0.220224,,1.16136,0.902695,1.92142,0.768428,1.29156,0.349661,-0.748747
2,-0.094892,0.636768,0.0552257,-0.281606,-0.0211867,0.452976,-0.157851,,-0.773996,1.04139,0.294318,-1.51213,-0.108865,2.26329,-0.690139
3,0.080628,-0.167331,-0.266373,-0.0806688,-0.0826383,-0.268307,0.0951453,,0.652677,-0.334166,-1.40226,-0.356954,-0.428702,-1.59989,0.370855
4,0.126338,0.160788,-0.131153,-0.229869,-0.164893,0.132993,-0.447996,,0.981006,0.411149,-0.604312,-1.08564,-0.816813,0.581818,-1.49206
Big,0.0382256,0.357449,-0.0457797,-0.0756579,0.203364,0.0501882,-0.526664,,0.600842,0.765341,-0.257547,-0.303554,0.561395,0.150071,-1.72004
,,,,h,,,,,,,,t(h),,,
small,0.373798,0.0615703,0.25193,0.336415,0.292907,-0.122174,-0.267287,,5.71379,0.325003,3.03399,4.53365,3.53261,-1.48733,-2.28172
2,0.285469,0.729702,0.252629,0.215264,-0.00186029,0.0725043,-0.230456,,5.84633,2.99636,3.38045,2.90225,-0.0240006,0.909587,-2.52984
3,0.34575,0.28827,0.235218,0.21357,0.0722335,-0.0628927,-0.320495,,7.02732,1.44544,3.10904,2.37281,0.940869,-0.941617,-3.13656


## 5.Volatility (r-Var portfolios)

In [110]:
dt3=rVarPortXs.mean()
dt3
dt3sd=rVarPortXs.std()
dt3sd
# List of tuples
students = [ (' ', '  ' , 'Mean' , ' ',' ',' ',' ',' ','SD',' ', ' '),
             (dt3[0], dt3[1], dt3[2] , dt3[3],dt3[4],' ',dt3sd[0], dt3sd[1], dt3sd[2] , dt3sd[3],dt3sd[4]) ,
             (dt3[5], dt3[6], dt3[7] , dt3[8],dt3[9],' ',dt3sd[5], dt3sd[6], dt3sd[7] , dt3sd[8],dt3sd[9]) ,
             (dt3[10], dt3[11], dt3[12] , dt3[13],dt3[14],' ',dt3sd[10], dt3sd[11], dt3sd[12] , dt3sd[13],dt3sd[14]) ,
             (dt3[15], dt3[16], dt3[17] , dt3[18],dt3[19],' ',dt3sd[15], dt3sd[16], dt3sd[17] , dt3sd[18],dt3sd[19]) ,
             (dt3[20], dt3[21], dt3[22] , dt3[23],dt3[24],' ',dt3sd[20], dt3sd[21], dt3sd[22] , dt3sd[23],dt3sd[24]) ]
# Create DataFrame object from a list of tuples
table3 = pd.DataFrame(students, columns = ['Low ' , '2', '3' , '4','High ',' ','Low ' , '2', '3' , '4','High '], index=['','small', '2', '3' , '4' , 'Big'])
table3#Pnael a )

,Low,2,3,4,High,,Low,2,3,4,High
,,,Mean,,,,,,SD,,
small,0.638616,0.540478,0.438742,0.586996,0.738323,,5.09967,6.22559,7.83518,8.47249,10.2896
2,0.556301,0.646622,0.623249,0.590322,0.573868,,5.16827,5.56182,6.50467,7.38655,8.51539
3,0.409507,0.613555,0.672454,0.372516,0.798048,,5.19055,5.37901,5.88477,6.05856,7.79928
4,0.724612,0.988633,0.880764,0.802325,0.608104,,4.72672,4.80626,4.86602,5.69497,6.58481
Big,1.05288,0.971946,0.847614,0.894251,1.0044,,4.19504,4.08572,4.45425,4.36514,4.96424


In [111]:
#B/M=6/7 OP= 9 INV=10 PRIOR B= 8
dt3a=rVarPort[6].mean()
dt3b=rVarPort[9].mean()
# List of tuples
students = [ (' ', '  ' , 'B/M' , ' ',' ',' ',' ',' ','OP',' ', ' '),
             (dt3a[0], dt3a[1], dt3a[2] , dt3a[3],dt3a[4],' ',dt3b[0], dt3b[1], dt3b[2] , dt3b[3],dt3b[4]) ,
             (dt3a[5], dt3a[6], dt3a[7] , dt3a[8],dt3a[9],' ',dt3b[5], dt3b[6], dt3b[7] , dt3b[8],dt3b[9]) ,
             (dt3a[10], dt3a[11], dt3a[12] , dt3a[13],dt3a[14],' ',dt3b[10], dt3b[11], dt3b[12] , dt3b[13],dt3b[14]) ,
             (dt3a[15], dt3a[16], dt3a[17] , dt3a[18],dt3a[19],' ',dt3b[15],dt3b[16], dt3b[17] , dt3b[18],dt3b[19]) ,
             (dt3a[20], dt3a[21], dt3a[22] , dt3a[23],dt3a[24],' ',dt3b[20], dt3b[21], dt3b[22] , dt3b[23],dt3b[24]) ]
# Create DataFrame object from a list of tuples
table3a = pd.DataFrame(students, columns = ['Low ' , '2', '3' , '4','High ',' ','Low ' , '2', '3' , '4','High '], index=['','small', '2', '3' , '4' , 'Big'])
table3a#Pnael a 


,Low,2,3,4,High,,Low,2,3,4,High
,,,B/M,,,,,,OP,,
small,0.703235,0.652228,0.637729,0.632861,0.701251,,0.197442,0.1086,0.0387725,-6.48442,-17.5466
2,0.593086,0.507203,0.483159,0.494893,0.496577,,0.262219,0.333425,0.323546,0.160326,-7.52476
3,0.581965,0.462567,0.422767,0.40387,0.439113,,0.248719,0.35053,0.371477,0.290961,-0.0134986
4,0.426591,0.405483,0.374749,0.375209,0.379228,,0.450393,0.528041,0.499322,0.445854,0.416903
Big,0.384546,0.344259,0.317757,0.299743,0.289606,,0.577857,0.848938,1.0639,1.14162,0.846043


In [112]:
#B/M=6/7 OP= 9 INV=10 PRIOR B= 8
dt3a=rVarPort[10].mean()
dt3b=rVarPort[8].mean()
# List of tuples
students = [ (' ', '  ' , 'Inv' , ' ',' ',' ',' ',' ','RVar',' ', ' '),
             (dt3a[0], dt3a[1], dt3a[2] , dt3a[3],dt3a[4],' ',dt3b[0], dt3b[1], dt3b[2] , dt3b[3],dt3b[4]) ,
             (dt3a[5], dt3a[6], dt3a[7] , dt3a[8],dt3a[9],' ',dt3b[5], dt3b[6], dt3b[7] , dt3b[8],dt3b[9]) ,
             (dt3a[10], dt3a[11], dt3a[12] , dt3a[13],dt3a[14],' ',dt3b[10], dt3b[11], dt3b[12] , dt3b[13],dt3b[14]) ,
             (dt3a[15], dt3a[16], dt3a[17] , dt3a[18],dt3a[19],' ',dt3b[15],dt3b[16], dt3b[17] , dt3b[18],dt3b[19]) ,
             (dt3a[20], dt3a[21], dt3a[22] , dt3a[23],dt3a[24],' ',dt3b[20], dt3b[21], dt3b[22] , dt3b[23],dt3b[24]) ]
# Create DataFrame object from a list of tuples
table3b = pd.DataFrame(students, columns = ['Low' , '2', '3' , '4','High',' ','Low' , '2', '3' , '4','High'], index=['','small', '2', '3' , '4' , 'Big'])
table3b#Pnael a 

,Low,2,3,4,High,,Low,2,3,4,High
,,,Inv,,,,,,RVar,,
small,0.101436,0.115561,0.159686,0.231058,0.236216,,2.37414,5.55601,9.22261,15.7285,76.6147
2,0.0838957,0.095658,0.130278,0.176742,0.276052,,1.36979,2.7074,4.30336,7.25452,24.6185
3,0.0807043,0.0862826,0.102206,0.140616,0.231864,,1.05432,1.83726,2.77259,4.53477,14.2192
4,0.0614478,0.0754594,0.0789826,0.11183,0.185075,,0.821284,1.34916,1.9517,2.99223,8.65314
Big,0.0526464,0.0676725,0.0835319,0.0946072,0.139896,,0.558933,0.968129,1.33495,1.97491,4.70341


### Regressions

In [113]:
result3a=regCAPM(rVarPortXs,F2)
# List of tuples
regcapmdata = [ ('', '  ' , 'a' , ' ',' ',' ',' ',' ','t(a)',' ', ' '),
             (result3a[0][0], result3a[0][1], result3a[0][2] , result3a[0][3],result3a[0][4],' ',result3a[2][0], result3a[2][1], result3a[2][2] , result3a[2][3],result3a[2][4]) ,
             (result3a[0][5], result3a[0][6], result3a[0][7] , result3a[0][8],result3a[0][9],' ',result3a[2][5], result3a[2][6], result3a[2][7] , result3a[2][8],result3a[2][9]) ,
             (result3a[0][10], result3a[0][11], result3a[0][12] , result3a[0][13],result3a[0][14],' ',result3a[2][10], result3a[2][11], result3a[2][12] , result3a[2][13],result3a[2][14]) ,
             (result3a[0][15], result3a[0][16], result3a[0][17] , result3a[0][18],result3a[0][19],' ',result3a[2][15], result3a[2][16], result3a[2][17] , result3a[2][18],result3a[2][19]) ,
             (result3a[0][20], result3a[0][21], result3a[0][22] , result3a[0][23],result3a[0][24],' ',result3a[2][20], result3a[2][21], result3a[2][22] , result3a[2][23],result3a[2][24]),
             ]
# Create DataFrame object from a list of tuples
table4a = pd.DataFrame(regcapmdata, columns = ['Low' , '2', '3' , '4','High',' ','Low' , '2', '3' , '4','High'], index=['','small', '2', '3' , '4' , 'Big'])
table4a

,Low,2,3,4,High,,Low,2,3,4,High
,,,a,,,,,,t(a),,
small,0.647481,0.223775,-0.174875,-0.346312,-0.285126,,3.91106,1.18008,-0.585957,-0.877717,-0.399378
2,0.441486,0.168828,0.0282026,-0.339979,-0.55486,,2.24314,1.22747,0.194156,-1.65172,-1.61409
3,0.0996507,-0.0360704,-0.120301,-0.332437,-0.212799,,0.531234,-0.21051,-0.74006,-1.95108,-0.619476
4,0.0754702,0.118896,0.16165,-0.0311768,-0.524842,,0.379383,0.703294,1.17504,-0.18004,-1.83998
Big,0.113279,0.0769576,-0.167334,0.0115382,-0.0674027,,0.835626,0.634848,-1.0342,0.0694573,-0.350386


In [114]:
result3a=regFivefactor(rVarPortXs,test) #For five factor regression
# List of tuples
reg5Fdata = [ ('', '  ' , 'a' , ' ',' ',' ',' ',' ','t(a)',' ', ' '),
             (result3a[0][0], result3a[0][1], result3a[0][2] , result3a[0][3],result3a[0][4],' ',result3a[2][0], result3a[2][1], result3a[2][2] , result3a[2][3],result3a[2][4]) ,
             (result3a[0][5], result3a[0][6], result3a[0][7] , result3a[0][8],result3a[0][9],' ',result3a[2][5], result3a[2][6], result3a[2][7] , result3a[2][8],result3a[2][9]) ,
             (result3a[0][10], result3a[0][11], result3a[0][12] , result3a[0][13],result3a[0][14],' ',result3a[2][10], result3a[2][11], result3a[2][12] , result3a[2][13],result3a[2][14]) ,
             (result3a[0][15], result3a[0][16], result3a[0][17] , result3a[0][18],result3a[0][19],' ',result3a[2][15], result3a[2][16], result3a[2][17] , result3a[2][18],result3a[2][19]) ,
             (result3a[0][20], result3a[0][21], result3a[0][22] , result3a[0][23],result3a[0][24],' ',result3a[2][20], result3a[2][21], result3a[2][22] , result3a[2][23],result3a[2][24]),
             ('', '  ' , 'b' , ' ',' ',' ',' ',' ','t(b)',' ', ' '),
             (result3a[1][0], result3a[1][1], result3a[1][2] , result3a[1][3],result3a[1][4],' ',result3a[3][0], result3a[3][1], result3a[3][2] , result3a[3][3],result3a[3][4]) ,
             (result3a[1][5], result3a[1][6], result3a[1][7] , result3a[1][8],result3a[1][9],' ',result3a[3][5], result3a[3][6], result3a[3][7] , result3a[3][8],result3a[3][9]) ,
             (result3a[1][10], result3a[1][11], result3a[1][12] , result3a[1][13],result3a[1][14],' ',result3a[3][10], result3a[3][11], result3a[3][12] , result3a[3][13],result3a[3][14]) ,
             (result3a[1][15], result3a[1][16], result3a[1][17] , result3a[1][18],result3a[1][19],' ',result3a[3][15], result3a[3][16], result3a[3][17] , result3a[3][18],result3a[3][19]) ,
             (result3a[1][20], result3a[1][21], result3a[1][22] , result3a[1][23],result3a[1][24],' ',result3a[3][20], result3a[3][21], result3a[3][22] , result3a[3][23],result3a[3][24]),
             ('', '  ' , 's' , ' ',' ',' ',' ',' ','t(s)',' ', ' '),
             (result3a[4][0], result3a[4][1], result3a[4][2] , result3a[4][3],result3a[4][4],' ',result3a[5][0], result3a[5][1], result3a[5][2] , result3a[5][3],result3a[5][4]),
             (result3a[4][5], result3a[4][6], result3a[4][7] , result3a[4][8],result3a[4][9],' ',result3a[5][5], result3a[5][6], result3a[5][7] , result3a[5][8],result3a[5][9]),
             (result3a[4][10], result3a[4][11], result3a[4][12] , result3a[4][13],result3a[4][14],' ',result3a[5][10], result3a[5][11], result3a[5][12] , result3a[5][13],result3a[5][14]),
             (result3a[4][15], result3a[4][16], result3a[4][17] , result3a[4][18],result3a[4][19],' ',result3a[5][15], result3a[5][16], result3a[5][17] , result3a[5][18],result3a[5][19]),
             (result3a[4][20], result3a[4][21], result3a[4][22] , result3a[4][23],result3a[4][24],' ',result3a[5][20], result3a[5][21], result3a[5][22] , result3a[5][23],result3a[5][24]),
             ('', '  ' , 'h' , ' ',' ',' ',' ',' ','t(h)',' ', ' '),
             (result3a[6][0], result3a[6][1], result3a[6][2] , result3a[6][3],result3a[6][4],' ',result3a[7][0], result3a[7][1], result3a[7][2] , result3a[7][3],result3a[7][4]) ,
             (result3a[6][5], result3a[6][6], result3a[6][7] , result3a[6][8],result3a[6][9],' ',result3a[7][5], result3a[7][6], result3a[7][7] , result3a[7][8],result3a[7][9]) ,
             (result3a[6][10], result3a[6][11], result3a[6][12] , result3a[6][13],result3a[6][14],' ',result3a[7][10], result3a[7][11], result3a[7][12] , result3a[7][13],result3a[7][14]) ,
             (result3a[6][15], result3a[6][16], result3a[6][17] , result3a[6][18],result3a[6][19],' ',result3a[7][15], result3a[7][16], result3a[7][17] , result3a[7][18],result3a[7][19]) ,
             (result3a[6][20], result3a[6][21], result3a[6][22] , result3a[6][23],result3a[6][24],' ',result3a[7][20], result3a[7][21], result3a[7][22] , result3a[7][23],result3a[7][24]), 
             ('', '  ' , 'r' , ' ',' ',' ',' ',' ','t(r)',' ', ' '),
             (result3a[8][0], result3a[8][1], result3a[8][2] , result3a[8][3],result3a[8][4],' ',result3a[9][0], result3a[9][1], result3a[9][2] , result3a[9][3],result3a[9][4]) ,
             (result3a[8][5], result3a[8][6], result3a[8][7] , result3a[8][8],result3a[8][9],' ',result3a[9][5], result3a[9][6], result3a[9][7] , result3a[9][8],result3a[9][9]) ,
             (result3a[8][10], result3a[8][11], result3a[8][12] , result3a[8][13],result3a[8][14],' ',result3a[9][10], result3a[9][11], result3a[9][12] , result3a[9][13],result3a[9][14]) ,
             (result3a[8][15], result3a[8][16], result3a[8][17] , result3a[8][18],result3a[8][19],' ',result3a[9][15], result3a[9][16], result3a[9][17] , result3a[9][18],result3a[9][19]) ,
             (result3a[8][20], result3a[8][21], result3a[8][22] , result3a[8][23],result3a[8][24],' ',result3a[9][20], result3a[9][21], result3a[9][22] , result3a[9][23],result3a[9][24]),
             ('', '  ' , 'c' , ' ',' ',' ',' ',' ','t(c)',' ', ' '),
             (result3a[10][0], result3a[10][1], result3a[10][2] , result3a[10][3],result3a[10][4],' ',result3a[11][0], result3a[11][1], result3a[11][2] , result3a[11][3],result3a[11][4]) ,
             (result3a[10][5], result3a[10][6], result3a[10][7] , result3a[10][8],result3a[10][9],' ',result3a[11][5], result3a[11][6], result3a[11][7] , result3a[11][8],result3a[11][9]) ,
             (result3a[10][10], result3a[10][11], result3a[10][12] , result3a[10][13],result3a[10][14],' ',result3a[11][10], result3a[11][11], result3a[11][12] , result3a[11][13],result3a[11][14]) ,
             (result3a[10][15], result3a[10][16], result3a[10][17] , result3a[10][18],result3a[10][19],' ',result3a[11][15], result3a[11][16], result3a[11][17] , result3a[11][18],result3a[11][19]) ,
             (result3a[10][20], result3a[10][21], result3a[10][22] , result3a[10][23],result3a[10][24],' ',result3a[11][20], result3a[11][21], result3a[11][22] , result3a[11][23],result3a[11][24])]



# Create DataFrame object from a list of tuples
table4b = pd.DataFrame(reg5Fdata, columns = ['Low β' , '2', '3' , '4','High',' ','Low' , '2', '3' , '4','High'], index=['','small', '2', '3' , '4' , 'Big','','small', '2', '3' , '4' , 'Big','','small', '2', '3' , '4' , 'Big','','small', '2', '3' , '4' , 'Big','','small', '2', '3' , '4' , 'Big','','small', '2', '3' , '4' , 'Big'])
table4b

,Low β,2,3,4,High,,Low,2,3,4,High
,,,a,,,,,,t(a),,
small,0.644401,0.197403,-0.177552,-0.333801,-0.165291,,4.021,1.06268,-0.585944,-0.833946,-0.25554
2,0.448043,0.171886,0.0249367,-0.323458,-0.490439,,2.25148,1.23953,0.170401,-1.55992,-1.51127
3,0.115166,-0.033119,-0.120396,-0.349972,-0.181135,,0.614923,-0.19239,-0.763971,-2.04582,-0.529937
4,0.0913974,0.122054,0.170033,-0.0258467,-0.50817,,0.486711,0.721288,1.31569,-0.153536,-1.7754
Big,0.100952,0.0759612,-0.18683,0.0287188,-0.0500931,,0.738995,0.61656,-1.18484,0.176501,-0.261102
,,,b,,,,,,t(b),,
small,0.719149,0.904455,1.21705,1.38192,1.66798,,18.0764,19.6132,16.1791,13.9074,10.3876
2,0.809255,0.980909,1.10395,1.266,1.44963,,16.3813,28.4943,30.3878,24.5942,17.9941
3,0.914417,1.00982,1.10934,1.07026,1.3163,,19.6678,23.63,28.356,25.2021,15.5128


## 6. Accrual 

In [115]:
dt3=acPortXs.mean()
dt3
dt3sd=acPortXs.std()
dt3sd
# List of tuples
students = [ (' ', '  ' , 'Mean' , ' ',' ',' ',' ',' ','SD',' ', ' '),
             (dt3[0], dt3[1], dt3[2] , dt3[3],dt3[4],' ',dt3sd[0], dt3sd[1], dt3sd[2] , dt3sd[3],dt3sd[4]) ,
             (dt3[5], dt3[6], dt3[7] , dt3[8],dt3[9],' ',dt3sd[5], dt3sd[6], dt3sd[7] , dt3sd[8],dt3sd[9]) ,
             (dt3[10], dt3[11], dt3[12] , dt3[13],dt3[14],' ',dt3sd[10], dt3sd[11], dt3sd[12] , dt3sd[13],dt3sd[14]) ,
             (dt3[15], dt3[16], dt3[17] , dt3[18],dt3[19],' ',dt3sd[15], dt3sd[16], dt3sd[17] , dt3sd[18],dt3sd[19]) ,
             (dt3[20], dt3[21], dt3[22] , dt3[23],dt3[24],' ',dt3sd[20], dt3sd[21], dt3sd[22] , dt3sd[23],dt3sd[24]) ]
# Create DataFrame object from a list of tuples
table3 = pd.DataFrame(students, columns = ['Low ' , '2', '3' , '4','High ',' ','Low ' , '2', '3' , '4','High '], index=['','small', '2', '3' , '4' , 'Big'])
table3#Pnael a )

,Low,2,3,4,High,,Low,2,3,4,High
,,,Mean,,,,,,SD,,
small,0.342487,0.889768,0.107552,0.509062,0.529203,,6.903,6.56965,6.29799,6.66758,7.01054
2,0.511977,0.473684,0.654172,0.678281,0.640849,,6.55968,6.18015,5.72151,6.15894,6.82789
3,0.752997,0.609877,0.827301,0.573901,0.633464,,5.97553,5.02255,5.25177,5.99455,6.06162
4,0.694671,0.870272,0.884264,0.866555,0.744774,,5.41473,4.98663,5.04171,5.18669,5.35478
Big,1.60472,1.08668,0.731413,0.882335,1.0536,,4.63615,4.11387,4.58359,4.19362,4.13066


In [116]:
#B/M=6/7 OP= 10 INV=11 AC= 9
dt3a=acPort[6].mean()
dt3b=acPort[10].mean()
# List of tuples
students = [ (' ', '  ' , 'B/M' , ' ',' ',' ',' ',' ','OP',' ', ' '),
             (dt3a[0], dt3a[1], dt3a[2] , dt3a[3],dt3a[4],' ',dt3b[0], dt3b[1], dt3b[2] , dt3b[3],dt3b[4]) ,
             (dt3a[5], dt3a[6], dt3a[7] , dt3a[8],dt3a[9],' ',dt3b[5], dt3b[6], dt3b[7] , dt3b[8],dt3b[9]) ,
             (dt3a[10], dt3a[11], dt3a[12] , dt3a[13],dt3a[14],' ',dt3b[10], dt3b[11], dt3b[12] , dt3b[13],dt3b[14]) ,
             (dt3a[15], dt3a[16], dt3a[17] , dt3a[18],dt3a[19],' ',dt3b[15],dt3b[16], dt3b[17] , dt3b[18],dt3b[19]) ,
             (dt3a[20], dt3a[21], dt3a[22] , dt3a[23],dt3a[24],' ',dt3b[20], dt3b[21], dt3b[22] , dt3b[23],dt3b[24]) ]
# Create DataFrame object from a list of tuples
table3a = pd.DataFrame(students, columns = ['Low ' , '2', '3' , '4','High ',' ','Low ' , '2', '3' , '4','High '], index=['','small', '2', '3' , '4' , 'Big'])
table3a#Pnael a 



,Low,2,3,4,High,,Low,2,3,4,High
,,,B/M,,,,,,OP,,
small,0.549322,0.640797,0.660481,0.680177,0.517859,,-0.182236,-0.0227536,0.0213942,0.0160464,-22.3178
2,0.387242,0.484957,0.481299,0.470891,0.381326,,0.286597,0.151587,0.177572,0.169474,0.0905043
3,0.312674,0.429486,0.427117,0.435565,0.3588,,0.286123,0.226422,0.193849,0.266043,0.201604
4,0.299029,0.37287,0.353513,0.325212,0.28368,,0.722255,0.291135,0.279452,0.370899,0.75001
Big,0.169977,0.255801,0.340438,0.2951,0.195303,,1.2539,0.413309,0.335839,0.44981,1.22459


In [117]:
#B/M=6/7 OP= 10 INV=11 AC= 9
dt3a=acPort[11].mean()
dt3b=acPort[9].mean()
# List of tuples
students = [ (' ', '  ' , 'Inv' , ' ',' ',' ',' ',' ','AC',' ', ' '),
             (dt3a[0], dt3a[1], dt3a[2] , dt3a[3],dt3a[4],' ',dt3b[0], dt3b[1], dt3b[2] , dt3b[3],dt3b[4]) ,
             (dt3a[5], dt3a[6], dt3a[7] , dt3a[8],dt3a[9],' ',dt3b[5], dt3b[6], dt3b[7] , dt3b[8],dt3b[9]) ,
             (dt3a[10], dt3a[11], dt3a[12] , dt3a[13],dt3a[14],' ',dt3b[10], dt3b[11], dt3b[12] , dt3b[13],dt3b[14]) ,
             (dt3a[15], dt3a[16], dt3a[17] , dt3a[18],dt3a[19],' ',dt3b[15],dt3b[16], dt3b[17] , dt3b[18],dt3b[19]) ,
             (dt3a[20], dt3a[21], dt3a[22] , dt3a[23],dt3a[24],' ',dt3b[20], dt3b[21], dt3b[22] , dt3b[23],dt3b[24]) ]
# Create DataFrame object from a list of tuples
table3b = pd.DataFrame(students, columns = ['Low' , '2', '3' , '4','High',' ','Low' , '2', '3' , '4','High'], index=['','small', '2', '3' , '4' , 'Big'])
table3b#Pnael a 

,Low,2,3,4,High,,Low,2,3,4,High
,,,Inv,,,,,,AC,,
small,0.105561,0.118414,0.114333,0.111061,0.268719,,-474.178,-3.36585,0.173006,3.5001,1100.84
2,0.13729,0.0980014,0.114038,0.151564,0.24147,,-30.9191,-3.26847,0.124074,3.45708,59.4308
3,0.0942174,0.112128,0.122435,0.112819,0.178739,,-31.0393,-3.38709,0.155258,3.45942,48.0342
4,0.0773101,0.0988928,0.111035,0.122161,0.172465,,-83.5089,-3.2271,0.126743,3.31419,35.765
Big,0.10183,0.0897522,0.081942,0.10407,0.0823609,,-28.028,-3.29757,0.10077,3.11814,28.3807


### Regressions

### Panel A: Regression Intercepts

#### Three Factor 

In [118]:
result3a=regCAPM(acPortXs,F2) #MKT,SMB,HML
# List of tuples
regcapmdata = [ ('', '  ' , 'a' , ' ',' ',' ',' ',' ','t(a)',' ', ' '),
             (result3a[0][0], result3a[0][1], result3a[0][2] , result3a[0][3],result3a[0][4],' ',result3a[2][0], result3a[2][1], result3a[2][2] , result3a[2][3],result3a[2][4]) ,
             (result3a[0][5], result3a[0][6], result3a[0][7] , result3a[0][8],result3a[0][9],' ',result3a[2][5], result3a[2][6], result3a[2][7] , result3a[2][8],result3a[2][9]) ,
             (result3a[0][10], result3a[0][11], result3a[0][12] , result3a[0][13],result3a[0][14],' ',result3a[2][10], result3a[2][11], result3a[2][12] , result3a[2][13],result3a[2][14]) ,
             (result3a[0][15], result3a[0][16], result3a[0][17] , result3a[0][18],result3a[0][19],' ',result3a[2][15], result3a[2][16], result3a[2][17] , result3a[2][18],result3a[2][19]) ,
             (result3a[0][20], result3a[0][21], result3a[0][22] , result3a[0][23],result3a[0][24],' ',result3a[2][20], result3a[2][21], result3a[2][22] , result3a[2][23],result3a[2][24]),
             ]
# Create DataFrame object from a list of tuples
table4a = pd.DataFrame(regcapmdata, columns = ['Low' , '2', '3' , '4','High',' ','Low' , '2', '3' , '4','High'], index=['','small', '2', '3' , '4' , 'Big'])
table4a

,Low,2,3,4,High,,Low,2,3,4,High
,,,a,,,,,,t(a),,
small,-0.279267,0.334669,-0.333784,-0.0530365,-0.166715,,-0.973978,1.40032,-1.29419,-0.228954,-0.666866
2,-0.291976,-0.251792,0.0925943,-0.0306665,-0.0620954,,-1.43478,-1.62368,0.553725,-0.162921,-0.292834
3,-0.174548,-0.126668,0.112519,-0.200464,-0.192414,,-0.809573,-0.822984,0.601617,-1.13691,-0.90843
4,-0.338083,0.0347266,0.0287351,0.062249,-0.321755,,-1.67721,0.173135,0.16117,0.318822,-1.49512
Big,0.363394,-0.00246285,-0.279327,0.000276761,0.0632923,,2.32303,-0.0152676,-1.61332,0.00139511,0.376738


#### Four Factor

In [119]:
result3a=regFourFactor(acPortXs,F4) #MKT,SMB,HML,CMA
# List of tuples
regcapmdata = [ ('', '  ' , 'a' , ' ',' ',' ',' ',' ','t(a)',' ', ' '),
             (result3a[0][0], result3a[0][1], result3a[0][2] , result3a[0][3],result3a[0][4],' ',result3a[2][0], result3a[2][1], result3a[2][2] , result3a[2][3],result3a[2][4]) ,
             (result3a[0][5], result3a[0][6], result3a[0][7] , result3a[0][8],result3a[0][9],' ',result3a[2][5], result3a[2][6], result3a[2][7] , result3a[2][8],result3a[2][9]) ,
             (result3a[0][10], result3a[0][11], result3a[0][12] , result3a[0][13],result3a[0][14],' ',result3a[2][10], result3a[2][11], result3a[2][12] , result3a[2][13],result3a[2][14]) ,
             (result3a[0][15], result3a[0][16], result3a[0][17] , result3a[0][18],result3a[0][19],' ',result3a[2][15], result3a[2][16], result3a[2][17] , result3a[2][18],result3a[2][19]) ,
             (result3a[0][20], result3a[0][21], result3a[0][22] , result3a[0][23],result3a[0][24],' ',result3a[2][20], result3a[2][21], result3a[2][22] , result3a[2][23],result3a[2][24]),
             ]
# Create DataFrame object from a list of tuples
table4a = pd.DataFrame(regcapmdata, columns = ['Low' , '2', '3' , '4','High',' ','Low' , '2', '3' , '4','High'], index=['','small', '2', '3' , '4' , 'Big'])
table4a

,Low,2,3,4,High,,Low,2,3,4,High
,,,a,,,,,,t(a),,
small,-0.28272,0.324492,-0.339903,-0.0634258,-0.171149,,-0.979189,1.36811,-1.31352,-0.276382,-0.680853
2,-0.29426,-0.262971,0.0934227,-0.0389003,-0.0581476,,-1.43575,-1.76236,0.554233,-0.208429,-0.272791
3,-0.17325,-0.11943,0.119877,-0.199977,-0.189824,,-0.797237,-0.784659,0.644348,-1.12496,-0.890026
4,-0.335772,0.0488771,0.0348967,0.073155,-0.322231,,-1.65401,0.252733,0.196153,0.381992,-1.48516
Big,0.367005,0.00393024,-0.290139,-0.00871861,0.046222,,2.33767,0.0244978,-1.72067,-0.0443805,0.300843


#### Five Factor

In [120]:
result3a=regCAPM(acPortXs,test) #MKT,SMB,HMLO,RMW,CMA
# List of tuples
regcapmdata = [ ('', '  ' , 'a' , ' ',' ',' ',' ',' ','t(a)',' ', ' '),
             (result3a[0][0], result3a[0][1], result3a[0][2] , result3a[0][3],result3a[0][4],' ',result3a[2][0], result3a[2][1], result3a[2][2] , result3a[2][3],result3a[2][4]) ,
             (result3a[0][5], result3a[0][6], result3a[0][7] , result3a[0][8],result3a[0][9],' ',result3a[2][5], result3a[2][6], result3a[2][7] , result3a[2][8],result3a[2][9]) ,
             (result3a[0][10], result3a[0][11], result3a[0][12] , result3a[0][13],result3a[0][14],' ',result3a[2][10], result3a[2][11], result3a[2][12] , result3a[2][13],result3a[2][14]) ,
             (result3a[0][15], result3a[0][16], result3a[0][17] , result3a[0][18],result3a[0][19],' ',result3a[2][15], result3a[2][16], result3a[2][17] , result3a[2][18],result3a[2][19]) ,
             (result3a[0][20], result3a[0][21], result3a[0][22] , result3a[0][23],result3a[0][24],' ',result3a[2][20], result3a[2][21], result3a[2][22] , result3a[2][23],result3a[2][24]),
             ]
# Create DataFrame object from a list of tuples
table4a = pd.DataFrame(regcapmdata, columns = ['Low' , '2', '3' , '4','High',' ','Low' , '2', '3' , '4','High'], index=['','small', '2', '3' , '4' , 'Big'])
table4a

,Low,2,3,4,High,,Low,2,3,4,High
,,,a,,,,,,t(a),,
small,-0.26568,0.357052,-0.340765,-0.0627365,-0.14804,,-0.915526,1.5347,-1.30322,-0.270547,-0.590503
2,-0.265771,-0.243663,0.102533,-0.0429392,-0.0457722,,-1.32302,-1.65911,0.604683,-0.227849,-0.213612
3,-0.193705,-0.111407,0.120298,-0.212627,-0.198423,,-0.894334,-0.72746,0.639913,-1.193,-0.923002
4,-0.313203,0.0539093,0.0432096,0.0586216,-0.330547,,-1.55656,0.27615,0.241169,0.305635,-1.5111
Big,0.388381,-0.0223396,-0.320374,-0.0321319,0.0298314,,2.52074,-0.14383,-1.98024,-0.165504,0.195586


### Panel B: Regression slopes

#### 3 Factors

In [121]:
result3a=regCAPM(acPortXs,F2) #MKT,SMB,HML
# List of tuples
regcapmdata = [ ('', '  ' , 'h' , ' ',' ',' ',' ',' ','t(h)',' ', ' '),
             (result3a[6][0], result3a[6][1], result3a[6][2] , result3a[6][3],result3a[6][4],' ',result3a[7][0], result3a[7][1], result3a[7][2] , result3a[7][3],result3a[7][4]) ,
             (result3a[6][5], result3a[6][6], result3a[6][7] , result3a[6][8],result3a[6][9],' ',result3a[7][5], result3a[7][6], result3a[7][7] , result3a[7][8],result3a[7][9]) ,
             (result3a[6][10], result3a[6][11], result3a[6][12] , result3a[6][13],result3a[6][14],' ',result3a[7][10], result3a[7][11], result3a[7][12] , result3a[7][13],result3a[7][14]) ,
             (result3a[6][15], result3a[6][16], result3a[6][17] , result3a[6][18],result3a[6][19],' ',result3a[7][15], result3a[7][16], result3a[7][17] , result3a[7][18],result3a[7][19]) ,
             (result3a[6][20], result3a[6][21], result3a[6][22] , result3a[6][23],result3a[6][24],' ',result3a[7][20], result3a[7][21], result3a[7][22] , result3a[7][23],result3a[7][24]),
             ]
# Create DataFrame object from a list of tuples
table4a = pd.DataFrame(regcapmdata, columns = ['Low' , '2', '3' , '4','High',' ','Low' , '2', '3' , '4','High'], index=['','small', '2', '3' , '4' , 'Big'])
table4a

,Low,2,3,4,High,,Low,2,3,4,High
,,,h,,,,,,t(h),,
small,0.00495821,-0.100275,0.0505917,-0.0265961,-0.170255,,0.0531511,-1.28962,0.602935,-0.352898,-2.09326
2,-0.157343,0.0164947,0.0461657,0.0369302,-0.0688367,,-2.37654,0.326934,0.848568,0.603051,-0.997793
3,-0.107548,-0.0398527,-0.0972214,0.0118491,-0.0779108,,-1.53321,-0.795864,-1.59777,0.206554,-1.1306
4,-0.0605839,0.0425058,-0.0367141,0.0311535,-0.226558,,-0.923801,0.651371,-0.632941,0.490434,-3.23583
Big,-0.16589,-0.171198,-0.0401261,-0.0500934,-0.156599,,-3.25953,-3.26204,-0.712348,-0.776141,-2.86506


#### 4 Factors

In [122]:
result3a=regFourFactor(acPortXs,F4) #For five factor regression
# List of tuples
reg5Fdata = [ ('', '  ' , 'h' , ' ',' ',' ',' ',' ','t(h)',' ', ' '),
             (result3a[6][0], result3a[6][1], result3a[6][2] , result3a[6][3],result3a[6][4],' ',result3a[7][0], result3a[7][1], result3a[7][2] , result3a[7][3],result3a[7][4]) ,
             (result3a[6][5], result3a[6][6], result3a[6][7] , result3a[6][8],result3a[6][9],' ',result3a[7][5], result3a[7][6], result3a[7][7] , result3a[7][8],result3a[7][9]) ,
             (result3a[6][10], result3a[6][11], result3a[6][12] , result3a[6][13],result3a[6][14],' ',result3a[7][10], result3a[7][11], result3a[7][12] , result3a[7][13],result3a[7][14]) ,
             (result3a[6][15], result3a[6][16], result3a[6][17] , result3a[6][18],result3a[6][19],' ',result3a[7][15], result3a[7][16], result3a[7][17] , result3a[7][18],result3a[7][19]) ,
             (result3a[6][20], result3a[6][21], result3a[6][22] , result3a[6][23],result3a[6][24],' ',result3a[7][20], result3a[7][21], result3a[7][22] , result3a[7][23],result3a[7][24]), 
             ('', '  ' , 'c' , ' ',' ',' ',' ',' ','t(c)',' ', ' '),
             (result3a[8][0], result3a[8][1], result3a[8][2] , result3a[8][3],result3a[8][4],' ',result3a[9][0], result3a[9][1], result3a[9][2] , result3a[9][3],result3a[9][4]) ,
             (result3a[8][5], result3a[8][6], result3a[8][7] , result3a[8][8],result3a[8][9],' ',result3a[9][5], result3a[9][6], result3a[9][7] , result3a[9][8],result3a[9][9]) ,
             (result3a[8][10], result3a[8][11], result3a[8][12] , result3a[8][13],result3a[8][14],' ',result3a[9][10], result3a[9][11], result3a[9][12] , result3a[9][13],result3a[9][14]) ,
             (result3a[8][15], result3a[8][16], result3a[8][17] , result3a[8][18],result3a[8][19],' ',result3a[9][15], result3a[9][16], result3a[9][17] , result3a[9][18],result3a[9][19]) ,
             (result3a[8][20], result3a[8][21], result3a[8][22] , result3a[8][23],result3a[8][24],' ',result3a[9][20], result3a[9][21], result3a[9][22] , result3a[9][23],result3a[9][24])]
            


# Create DataFrame object from a list of tuples
table4b = pd.DataFrame(reg5Fdata, columns = ['Low β' , '2', '3' , '4','High',' ','Low' , '2', '3' , '4','High'], index=['','small', '2', '3' , '4' , 'Big','','small', '2', '3' , '4' , 'Big'])
table4b

,Low β,2,3,4,High,,Low,2,3,4,High
,,,h,,,,,,t(h),,
small,-0.02144,-0.178077,0.00381408,-0.106018,-0.204154,,-0.02144,-0.178077,0.00381408,-0.106018,-0.204154
2,-0.174808,-0.0689631,0.0524986,-0.0260138,-0.0386578,,-0.174808,-0.0689631,0.0524986,-0.0260138,-0.0386578
3,-0.0976223,0.0154815,-0.0409679,0.0155679,-0.0581106,,-0.0976223,0.0154815,-0.0409679,0.0155679,-0.0581106
4,-0.0429178,0.15068,0.010389,0.114525,-0.230193,,-0.0429178,0.15068,0.010389,0.114525,-0.230193
Big,-0.138284,-0.122325,-0.122777,-0.118859,-0.287093,,-0.138284,-0.122325,-0.122777,-0.118859,-0.287093
,,,c,,,,,,t(c),,
small,0.0835805,0.246332,0.148105,0.251462,0.107327,,0.399704,1.43404,0.790266,1.513,0.589537
2,0.0552942,0.270572,-0.0200509,0.19929,-0.0955505,,0.372522,2.50377,-0.164247,1.4744,-0.618948
3,-0.0314262,-0.175196,-0.178106,-0.011774,-0.0626903,,-0.199678,-1.58934,-1.32187,-0.0914542,-0.40586


#### 5 Factors

In [123]:
result3a=regFivefactor(acPortXs,test) #For five factor regression
# List of tuples
reg5Fdata = [ ('', '  ' , 'a' , ' ',' ',' ',' ',' ','t(a)',' ', ' '),
             (result3a[0][0], result3a[0][1], result3a[0][2] , result3a[0][3],result3a[0][4],' ',result3a[2][0], result3a[2][1], result3a[2][2] , result3a[2][3],result3a[2][4]) ,
             (result3a[0][5], result3a[0][6], result3a[0][7] , result3a[0][8],result3a[0][9],' ',result3a[2][5], result3a[2][6], result3a[2][7] , result3a[2][8],result3a[2][9]) ,
             (result3a[0][10], result3a[0][11], result3a[0][12] , result3a[0][13],result3a[0][14],' ',result3a[2][10], result3a[2][11], result3a[2][12] , result3a[2][13],result3a[2][14]) ,
             (result3a[0][15], result3a[0][16], result3a[0][17] , result3a[0][18],result3a[0][19],' ',result3a[2][15], result3a[2][16], result3a[2][17] , result3a[2][18],result3a[2][19]) ,
             (result3a[0][20], result3a[0][21], result3a[0][22] , result3a[0][23],result3a[0][24],' ',result3a[2][20], result3a[2][21], result3a[2][22] , result3a[2][23],result3a[2][24]),
             ('', '  ' , 'b' , ' ',' ',' ',' ',' ','t(b)',' ', ' '),
             (result3a[1][0], result3a[1][1], result3a[1][2] , result3a[1][3],result3a[1][4],' ',result3a[3][0], result3a[3][1], result3a[3][2] , result3a[3][3],result3a[3][4]) ,
             (result3a[1][5], result3a[1][6], result3a[1][7] , result3a[1][8],result3a[1][9],' ',result3a[3][5], result3a[3][6], result3a[3][7] , result3a[3][8],result3a[3][9]) ,
             (result3a[1][10], result3a[1][11], result3a[1][12] , result3a[1][13],result3a[1][14],' ',result3a[3][10], result3a[3][11], result3a[3][12] , result3a[3][13],result3a[3][14]) ,
             (result3a[1][15], result3a[1][16], result3a[1][17] , result3a[1][18],result3a[1][19],' ',result3a[3][15], result3a[3][16], result3a[3][17] , result3a[3][18],result3a[3][19]) ,
             (result3a[1][20], result3a[1][21], result3a[1][22] , result3a[1][23],result3a[1][24],' ',result3a[3][20], result3a[3][21], result3a[3][22] , result3a[3][23],result3a[3][24]),
             ('', '  ' , 's' , ' ',' ',' ',' ',' ','t(s)',' ', ' '),
             (result3a[4][0], result3a[4][1], result3a[4][2] , result3a[4][3],result3a[4][4],' ',result3a[5][0], result3a[5][1], result3a[5][2] , result3a[5][3],result3a[5][4]),
             (result3a[4][5], result3a[4][6], result3a[4][7] , result3a[4][8],result3a[4][9],' ',result3a[5][5], result3a[5][6], result3a[5][7] , result3a[5][8],result3a[5][9]),
             (result3a[4][10], result3a[4][11], result3a[4][12] , result3a[4][13],result3a[4][14],' ',result3a[5][10], result3a[5][11], result3a[5][12] , result3a[5][13],result3a[5][14]),
             (result3a[4][15], result3a[4][16], result3a[4][17] , result3a[4][18],result3a[4][19],' ',result3a[5][15], result3a[5][16], result3a[5][17] , result3a[5][18],result3a[5][19]),
             (result3a[4][20], result3a[4][21], result3a[4][22] , result3a[4][23],result3a[4][24],' ',result3a[5][20], result3a[5][21], result3a[5][22] , result3a[5][23],result3a[5][24]),
             ('', '  ' , 'h' , ' ',' ',' ',' ',' ','t(h)',' ', ' '),
             (result3a[6][0], result3a[6][1], result3a[6][2] , result3a[6][3],result3a[6][4],' ',result3a[7][0], result3a[7][1], result3a[7][2] , result3a[7][3],result3a[7][4]) ,
             (result3a[6][5], result3a[6][6], result3a[6][7] , result3a[6][8],result3a[6][9],' ',result3a[7][5], result3a[7][6], result3a[7][7] , result3a[7][8],result3a[7][9]) ,
             (result3a[6][10], result3a[6][11], result3a[6][12] , result3a[6][13],result3a[6][14],' ',result3a[7][10], result3a[7][11], result3a[7][12] , result3a[7][13],result3a[7][14]) ,
             (result3a[6][15], result3a[6][16], result3a[6][17] , result3a[6][18],result3a[6][19],' ',result3a[7][15], result3a[7][16], result3a[7][17] , result3a[7][18],result3a[7][19]) ,
             (result3a[6][20], result3a[6][21], result3a[6][22] , result3a[6][23],result3a[6][24],' ',result3a[7][20], result3a[7][21], result3a[7][22] , result3a[7][23],result3a[7][24]), 
             ('', '  ' , 'r' , ' ',' ',' ',' ',' ','t(r)',' ', ' '),
             (result3a[8][0], result3a[8][1], result3a[8][2] , result3a[8][3],result3a[8][4],' ',result3a[9][0], result3a[9][1], result3a[9][2] , result3a[9][3],result3a[9][4]) ,
             (result3a[8][5], result3a[8][6], result3a[8][7] , result3a[8][8],result3a[8][9],' ',result3a[9][5], result3a[9][6], result3a[9][7] , result3a[9][8],result3a[9][9]) ,
             (result3a[8][10], result3a[8][11], result3a[8][12] , result3a[8][13],result3a[8][14],' ',result3a[9][10], result3a[9][11], result3a[9][12] , result3a[9][13],result3a[9][14]) ,
             (result3a[8][15], result3a[8][16], result3a[8][17] , result3a[8][18],result3a[8][19],' ',result3a[9][15], result3a[9][16], result3a[9][17] , result3a[9][18],result3a[9][19]) ,
             (result3a[8][20], result3a[8][21], result3a[8][22] , result3a[8][23],result3a[8][24],' ',result3a[9][20], result3a[9][21], result3a[9][22] , result3a[9][23],result3a[9][24]),
             ('', '  ' , 'c' , ' ',' ',' ',' ',' ','t(c)',' ', ' '),
             (result3a[10][0], result3a[10][1], result3a[10][2] , result3a[10][3],result3a[10][4],' ',result3a[11][0], result3a[11][1], result3a[11][2] , result3a[11][3],result3a[11][4]) ,
             (result3a[10][5], result3a[10][6], result3a[10][7] , result3a[10][8],result3a[10][9],' ',result3a[11][5], result3a[11][6], result3a[11][7] , result3a[11][8],result3a[11][9]) ,
             (result3a[10][10], result3a[10][11], result3a[10][12] , result3a[10][13],result3a[10][14],' ',result3a[11][10], result3a[11][11], result3a[11][12] , result3a[11][13],result3a[11][14]) ,
             (result3a[10][15], result3a[10][16], result3a[10][17] , result3a[10][18],result3a[10][19],' ',result3a[11][15], result3a[11][16], result3a[11][17] , result3a[11][18],result3a[11][19]) ,
             (result3a[10][20], result3a[10][21], result3a[10][22] , result3a[10][23],result3a[10][24],' ',result3a[11][20], result3a[11][21], result3a[11][22] , result3a[11][23],result3a[11][24])]



# Create DataFrame object from a list of tuples
table4b = pd.DataFrame(reg5Fdata, columns = ['Low β' , '2', '3' , '4','High',' ','Low' , '2', '3' , '4','High'], index=['','small', '2', '3' , '4' , 'Big','','small', '2', '3' , '4' , 'Big','','small', '2', '3' , '4' , 'Big','','small', '2', '3' , '4' , 'Big','','small', '2', '3' , '4' , 'Big','','small', '2', '3' , '4' , 'Big'])
table4b

,Low β,2,3,4,High,,Low,2,3,4,High
,,,a,,,,,,t(a),,
small,-0.26568,0.357052,-0.340765,-0.0627365,-0.14804,,-0.915526,1.5347,-1.30322,-0.270547,-0.590503
2,-0.265771,-0.243663,0.102533,-0.0429392,-0.0457722,,-1.32302,-1.65911,0.604683,-0.227849,-0.213612
3,-0.193705,-0.111407,0.120298,-0.212627,-0.198423,,-0.894334,-0.72746,0.639913,-1.193,-0.923002
4,-0.313203,0.0539093,0.0432096,0.0586216,-0.330547,,-1.55656,0.27615,0.241169,0.305635,-1.5111
Big,0.388381,-0.0223396,-0.320374,-0.0321319,0.0298314,,2.52074,-0.14383,-1.98024,-0.165504,0.195586
,,,b,,,,,,t(b),,
small,1.09339,0.987689,0.931341,1.01777,1.0607,,15.1775,17.1012,14.3478,17.6801,17.0432
2,1.10352,1.13256,0.964331,1.09672,1.09137,,22.1286,31.0641,22.9089,23.4424,20.5169
3,1.08533,0.936663,0.900862,1.0821,1.06246,,20.1853,24.6374,19.3034,24.4571,19.9084


## 7.Momentum

In [124]:
dt3=momPortXs.mean()
dt3
dt3sd=momPortXs.std()
dt3sd
# List of tuples
students = [ (' ', '  ' , 'Mean' , ' ',' ',' ',' ',' ','SD',' ', ' '),
             (dt3[0], dt3[1], dt3[2] , dt3[3],dt3[4],' ',dt3sd[0], dt3sd[1], dt3sd[2] , dt3sd[3],dt3sd[4]) ,
             (dt3[5], dt3[6], dt3[7] , dt3[8],dt3[9],' ',dt3sd[5], dt3sd[6], dt3sd[7] , dt3sd[8],dt3sd[9]) ,
             (dt3[10], dt3[11], dt3[12] , dt3[13],dt3[14],' ',dt3sd[10], dt3sd[11], dt3sd[12] , dt3sd[13],dt3sd[14]) ,
             (dt3[15], dt3[16], dt3[17] , dt3[18],dt3[19],' ',dt3sd[15], dt3sd[16], dt3sd[17] , dt3sd[18],dt3sd[19]) ,
             (dt3[20], dt3[21], dt3[22] , dt3[23],dt3[24],' ',dt3sd[20], dt3sd[21], dt3sd[22] , dt3sd[23],dt3sd[24]) ]
# Create DataFrame object from a list of tuples
table3 = pd.DataFrame(students, columns = ['Low ' , '2', '3' , '4','High ',' ','Low ' , '2', '3' , '4','High '], index=['','small', '2', '3' , '4' , 'Big'])
table3#Pnael a )

,Low,2,3,4,High,,Low,2,3,4,High
,,,Mean,,,,,,SD,,
small,0.307971,0.341884,0.710725,0.641449,0.746377,,9.11815,6.12708,6.10712,5.41881,6.44785
2,0.372899,0.691884,0.651159,0.496087,0.750725,,8.55168,6.61953,5.72581,5.58432,6.58836
3,0.115942,0.447246,0.536377,0.39913,0.908406,,8.74706,6.64569,5.49135,5.23255,5.73969
4,0.170435,0.556232,0.71942,0.762319,1.06652,,9.04698,6.22553,5.33148,4.6424,5.10861
Big,0.273768,0.65,0.961739,0.72971,1.11087,,7.90741,5.0534,4.34495,4.0859,4.46934


In [125]:
#B/M=6/7 OP= 10 INV=11 AC= 9
dt3a=momPort[6].mean()
dt3b=momPort[10].mean()
# List of tuples
students = [ (' ', '  ' , 'B/M' , ' ',' ',' ',' ',' ','OP',' ', ' '),
             (dt3a[0], dt3a[1], dt3a[2] , dt3a[3],dt3a[4],' ',dt3b[0], dt3b[1], dt3b[2] , dt3b[3],dt3b[4]) ,
             (dt3a[5], dt3a[6], dt3a[7] , dt3a[8],dt3a[9],' ',dt3b[5], dt3b[6], dt3b[7] , dt3b[8],dt3b[9]) ,
             (dt3a[10], dt3a[11], dt3a[12] , dt3a[13],dt3a[14],' ',dt3b[10], dt3b[11], dt3b[12] , dt3b[13],dt3b[14]) ,
             (dt3a[15], dt3a[16], dt3a[17] , dt3a[18],dt3a[19],' ',dt3b[15],dt3b[16], dt3b[17] , dt3b[18],dt3b[19]) ,
             (dt3a[20], dt3a[21], dt3a[22] , dt3a[23],dt3a[24],' ',dt3b[20], dt3b[21], dt3b[22] , dt3b[23],dt3b[24]) ]
# Create DataFrame object from a list of tuples
table3a = pd.DataFrame(students, columns = ['Low ' , '2', '3' , '4','High ',' ','Low ' , '2', '3' , '4','High '], index=['','small', '2', '3' , '4' , 'Big'])
table3a#Pnael a 




KeyError: 10

In [126]:
#B/M=6/7 OP= 10 INV=11 AC= 9
dt3a=momPort[11].mean()
dt3b=momPort[6].mean()
# List of tuples
students = [ (' ', '  ' , 'Inv' , ' ',' ',' ',' ',' ','Prior2-12',' ', ' '),
             (dt3a[0], dt3a[1], dt3a[2] , dt3a[3],dt3a[4],' ',dt3b[0], dt3b[1], dt3b[2] , dt3b[3],dt3b[4]) ,
             (dt3a[5], dt3a[6], dt3a[7] , dt3a[8],dt3a[9],' ',dt3b[5], dt3b[6], dt3b[7] , dt3b[8],dt3b[9]) ,
             (dt3a[10], dt3a[11], dt3a[12] , dt3a[13],dt3a[14],' ',dt3b[10], dt3b[11], dt3b[12] , dt3b[13],dt3b[14]) ,
             (dt3a[15], dt3a[16], dt3a[17] , dt3a[18],dt3a[19],' ',dt3b[15],dt3b[16], dt3b[17] , dt3b[18],dt3b[19]) ,
             (dt3a[20], dt3a[21], dt3a[22] , dt3a[23],dt3a[24],' ',dt3b[20], dt3b[21], dt3b[22] , dt3b[23],dt3b[24]) ]
# Create DataFrame object from a list of tuples
table3b = pd.DataFrame(students, columns = ['Low' , '2', '3' , '4','High',' ','Low' , '2', '3' , '4','High'], index=['','small', '2', '3' , '4' , 'Big'])
table3b#Pnael a 

KeyError: 11

### Regression 

#### 5 Factors

In [127]:
result3a=regFivefactor(momPortXs,F6)
# List of tuples
regcapmdata = [ ('', '  ' , 'a' , ' ',' ',' ',' ',' ','t(a)',' ', ' '),
             (result3a[0][0], result3a[0][1], result3a[0][2] , result3a[0][3],result3a[0][4],' ',result3a[2][0], result3a[2][1], result3a[2][2] , result3a[2][3],result3a[2][4]) ,
             (result3a[0][5], result3a[0][6], result3a[0][7] , result3a[0][8],result3a[0][9],' ',result3a[2][5], result3a[2][6], result3a[2][7] , result3a[2][8],result3a[2][9]) ,
             (result3a[0][10], result3a[0][11], result3a[0][12] , result3a[0][13],result3a[0][14],' ',result3a[2][10], result3a[2][11], result3a[2][12] , result3a[2][13],result3a[2][14]) ,
             (result3a[0][15], result3a[0][16], result3a[0][17] , result3a[0][18],result3a[0][19],' ',result3a[2][15], result3a[2][16], result3a[2][17] , result3a[2][18],result3a[2][19]) ,
             (result3a[0][20], result3a[0][21], result3a[0][22] , result3a[0][23],result3a[0][24],' ',result3a[2][20], result3a[2][21], result3a[2][22] , result3a[2][23],result3a[2][24]),
             ]
# Create DataFrame object from a list of tuples
table4a = pd.DataFrame(regcapmdata, columns = ['Low' , '2', '3' , '4','High',' ','Low' , '2', '3' , '4','High'], index=['','small', '2', '3' , '4' , 'Big'])
table4a

,Low,2,3,4,High,,Low,2,3,4,High
,,,a,,,,,,t(a),,
small,-0.405524,0.00845709,0.523708,0.49882,0.13258,,-0.978249,0.0462503,2.04529,2.6277,0.554593
2,-0.423659,0.140986,0.256539,0.0759071,-0.068428,,-1.09029,0.799102,1.65728,0.395708,-0.273525
3,-0.567079,-0.160198,-0.00597893,-0.110385,0.00778818,,-1.15166,-0.780671,-0.039481,-0.681678,0.0339367
4,-0.452347,-0.158567,-0.0095325,-0.0185565,-0.00295407,,-0.893296,-0.715702,-0.0592511,-0.128086,-0.011742
Big,-0.204071,0.0659683,0.155495,-0.228439,-0.0849172,,-0.348137,0.237683,1.00675,-1.59334,-0.394955


#### 6 Factors

In [128]:
momtest=test.copy()
momtest['Mom']=FF6['Mom   ']

In [129]:
momtest

,Mkt-RF,SMB,HMLO,RMW,CMA,Mom
Date,,,,,,
2015-01,-3.11,-0.88,-1.345343,1.68,-1.65,3.81
2015-02,6.14,0.23,-0.779544,-1.16,-1.78,-2.83
2015-03,-1.12,3.04,-0.902406,0.10,-0.51,3.00
2015-04,0.59,-3.05,3.357457,-0.04,-0.45,-7.32
2015-05,1.36,0.81,-0.623699,-1.82,-0.74,6.03
...,...,...,...,...,...,...
2020-05,5.58,1.92,-3.154896,0.71,-3.28,0.67
2020-06,2.46,1.94,-3.613818,0.03,0.34,-0.75
2020-07,5.77,-3.03,-2.118145,0.55,1.06,7.61


In [130]:
result3a=regSixfactor(momPortXs,momtest)
# List of tuples
reg5Fdata = [ ('', '  ' , 'a' , ' ',' ',' ',' ',' ','t(a)',' ', ' '),
             (result3a[0][0], result3a[0][1], result3a[0][2] , result3a[0][3],result3a[0][4],' ',result3a[2][0], result3a[2][1], result3a[2][2] , result3a[2][3],result3a[2][4]) ,
             (result3a[0][5], result3a[0][6], result3a[0][7] , result3a[0][8],result3a[0][9],' ',result3a[2][5], result3a[2][6], result3a[2][7] , result3a[2][8],result3a[2][9]) ,
             (result3a[0][10], result3a[0][11], result3a[0][12] , result3a[0][13],result3a[0][14],' ',result3a[2][10], result3a[2][11], result3a[2][12] , result3a[2][13],result3a[2][14]) ,
             (result3a[0][15], result3a[0][16], result3a[0][17] , result3a[0][18],result3a[0][19],' ',result3a[2][15], result3a[2][16], result3a[2][17] , result3a[2][18],result3a[2][19]) ,
             (result3a[0][20], result3a[0][21], result3a[0][22] , result3a[0][23],result3a[0][24],' ',result3a[2][20], result3a[2][21], result3a[2][22] , result3a[2][23],result3a[2][24]),
             ('', '  ' , 'h' , ' ',' ',' ',' ',' ','t(h)',' ', ' '),
             (result3a[6][0], result3a[6][1], result3a[6][2] , result3a[6][3],result3a[6][4],' ',result3a[7][0], result3a[7][1], result3a[7][2] , result3a[7][3],result3a[7][4]) ,
             (result3a[6][5], result3a[6][6], result3a[6][7] , result3a[6][8],result3a[6][9],' ',result3a[7][5], result3a[7][6], result3a[7][7] , result3a[7][8],result3a[7][9]) ,
             (result3a[6][10], result3a[6][11], result3a[6][12] , result3a[6][13],result3a[6][14],' ',result3a[7][10], result3a[7][11], result3a[7][12] , result3a[7][13],result3a[7][14]) ,
             (result3a[6][15], result3a[6][16], result3a[6][17] , result3a[6][18],result3a[6][19],' ',result3a[7][15], result3a[7][16], result3a[7][17] , result3a[7][18],result3a[7][19]) ,
             (result3a[6][20], result3a[6][21], result3a[6][22] , result3a[6][23],result3a[6][24],' ',result3a[7][20], result3a[7][21], result3a[7][22] , result3a[7][23],result3a[7][24]), 
             ('', '  ' , 'r' , ' ',' ',' ',' ',' ','t(r)',' ', ' '),
             (result3a[8][0], result3a[8][1], result3a[8][2] , result3a[8][3],result3a[8][4],' ',result3a[9][0], result3a[9][1], result3a[9][2] , result3a[9][3],result3a[9][4]) ,
             (result3a[8][5], result3a[8][6], result3a[8][7] , result3a[8][8],result3a[8][9],' ',result3a[9][5], result3a[9][6], result3a[9][7] , result3a[9][8],result3a[9][9]) ,
             (result3a[8][10], result3a[8][11], result3a[8][12] , result3a[8][13],result3a[8][14],' ',result3a[9][10], result3a[9][11], result3a[9][12] , result3a[9][13],result3a[9][14]) ,
             (result3a[8][15], result3a[8][16], result3a[8][17] , result3a[8][18],result3a[8][19],' ',result3a[9][15], result3a[9][16], result3a[9][17] , result3a[9][18],result3a[9][19]) ,
             (result3a[8][20], result3a[8][21], result3a[8][22] , result3a[8][23],result3a[8][24],' ',result3a[9][20], result3a[9][21], result3a[9][22] , result3a[9][23],result3a[9][24]),
             ('', '  ' , 'c' , ' ',' ',' ',' ',' ','t(c)',' ', ' '),
             (result3a[10][0], result3a[10][1], result3a[10][2] , result3a[10][3],result3a[10][4],' ',result3a[11][0], result3a[11][1], result3a[11][2] , result3a[11][3],result3a[11][4]) ,
             (result3a[10][5], result3a[10][6], result3a[10][7] , result3a[10][8],result3a[10][9],' ',result3a[11][5], result3a[11][6], result3a[11][7] , result3a[11][8],result3a[11][9]) ,
             (result3a[10][10], result3a[10][11], result3a[10][12] , result3a[10][13],result3a[10][14],' ',result3a[11][10], result3a[11][11], result3a[11][12] , result3a[11][13],result3a[11][14]) ,
             (result3a[10][15], result3a[10][16], result3a[10][17] , result3a[10][18],result3a[10][19],' ',result3a[11][15], result3a[11][16], result3a[11][17] , result3a[11][18],result3a[11][19]) ,
             (result3a[10][20], result3a[10][21], result3a[10][22] , result3a[10][23],result3a[10][24],' ',result3a[11][20], result3a[11][21], result3a[11][22] , result3a[11][23],result3a[11][24]),
             ('', '  ' , 'm' , ' ',' ',' ',' ',' ','t(m)',' ', ' '),
             (result3a[12][0], result3a[12][1], result3a[12][2] , result3a[12][3],result3a[12][4],' ',result3a[13][0], result3a[13][1], result3a[13][2] , result3a[13][3],result3a[13][4]) ,
             (result3a[12][5], result3a[12][6], result3a[12][7] , result3a[12][8],result3a[12][9],' ',result3a[13][5], result3a[13][6], result3a[13][7] , result3a[13][8],result3a[13][9]) ,
             (result3a[12][10], result3a[12][11], result3a[12][12] , result3a[12][13],result3a[12][14],' ',result3a[13][10], result3a[13][11], result3a[13][12] , result3a[13][13],result3a[13][14]) ,
             (result3a[12][15], result3a[12][16], result3a[12][17] , result3a[12][18],result3a[12][19],' ',result3a[13][15], result3a[13][16], result3a[13][17] , result3a[13][18],result3a[13][19]) ,
             (result3a[12][20], result3a[12][21], result3a[12][22] , result3a[12][23],result3a[12][24],' ',result3a[13][20], result3a[13][21], result3a[13][22] , result3a[13][23],result3a[3][24])]



# Create DataFrame object from a list of tuples [Main Reference: samthorold / Thesis] 
table4b = pd.DataFrame(reg5Fdata, columns = ['Low' , '2', '3' , '4','High',' ','Low' , '2', '3' , '4','High'], index=['','small', '2', '3' , '4' , 'Big','','small', '2', '3' , '4' , 'Big','','small', '2', '3' , '4' , 'Big','','small', '2', '3' , '4' , 'Big','','small', '2', '3' , '4' , 'Big'])
table4b

,Low,2,3,4,High,,Low,2,3,4,High
,,,a,,,,,,t(a),,
small,-0.314524,0.0281875,0.542511,0.459598,0.104786,,-0.926674,0.159479,2.14082,2.8731,0.456795
2,-0.304551,0.163381,0.252583,0.0372229,-0.117821,,-1.34449,0.975016,1.62067,0.227942,-0.548643
3,-0.411809,-0.106711,-0.0114438,-0.138963,-0.0471323,,-1.5262,-0.715827,-0.0752298,-0.963399,-0.26409
4,-0.31565,-0.100775,-0.00297115,-0.034591,-0.0523928,,-0.888586,-0.626568,-0.018409,-0.247594,-0.242171
Big,-0.0419405,0.145997,0.173364,-0.240614,-0.133143,,-0.104823,0.813666,1.16912,-1.70724,-0.76512
,,,h,,,,,,t(h),,
small,-0.246217,0.152347,0.328023,0.536895,-0.11696,,-1.72493,2.04956,3.07792,7.98074,-1.21237
2,-0.342891,0.219917,0.257724,0.387656,0.0236482,,-3.59943,3.12068,3.93213,5.64471,0.261846
3,-0.277577,0.180485,0.346365,0.290617,-0.0499084,,-2.44613,2.87885,5.41419,4.79079,-0.664948
